In [33]:
# step1_prep_prices.py
from pathlib import Path
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import yfinance as yf
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

warnings.simplefilter(action="ignore", category=FutureWarning)

# --- Asset Universes ---
SEMIS = [
    "NVDA","AMD","AVGO","QCOM","TSM","ASML","LRCX","TXN","INTC","ADI",
    "MRVL","KLAC","LSCC","GFS","AMAT","ALGM","CRDO","RMBS",
    "TER","COHU","ENTG","ICHR","SNPS","CDNS","ANSS"
]

# --- Diversifier Universe ---
# US Treasury Bonds, Gold, Copper, Biotech, Vix, Investment Grade Credit

DIVERSIFIERS = ["TLT","GLD","CPER","USO","XBI","VIXY","LQD"]

# Synthetic "short" tickers (just aliases for testing diversifiying qualities of macro shorts)

SHORTS = {"SHORT_CPER": "CPER", "SHORT_USO": "USO", "SHORT_GLD": "GLD"}

# Extra tickers for custom factor construction as follows
# ETF to use as Benchmark for Semis (SMH)
# ETF to use as Benchmark for Software (SMH)
# ETF to use as Benchmark for Cloud (SKYY)
# Named stocks to form the Hyperscaler basket: Microsoft, Amazon, GOOGL

FACTOR_TICKERS = ["SMH", "IGV", "SKYY", "MSFT", "AMZN", "GOOGL", "SPY"]

# Combined ticker set for raw data pulls
TICKERS = SEMIS + DIVERSIFIERS + list(SHORTS.values()) + FACTOR_TICKERS

YEARS = 3
MIN_DAYS = 500
TRADING_DAYS = 252
OUTDIR = Path("outputs"); OUTDIR.mkdir(parents=True, exist_ok=True)

# --- Industry Classification mapping ---
CLASSIFICATION = {
    # Semiconductor Equipment
    "AMAT": ("Applied Materials", "Semiconductor Equipment", "Wafer Fab Equipment"),
    "LRCX": ("Lam Research", "Semiconductor Equipment", "Wafer Fab Equipment"),
    "KLAC": ("KLA", "Semiconductor Equipment", "Process Control"),
    "ASML": ("ASML Holding", "Semiconductor Equipment", "Lithography"),
    "TER": ("Teradyne", "Semiconductor Equipment", "Test Equipment"),
    "COHU": ("Cohu", "Semiconductor Equipment", "Test Equipment"),
    "ENTG": ("Entegris", "Semiconductor Equipment", "Materials/Consumables"),
    "ICHR": ("Ichor", "Semiconductor Equipment", "Subsystems"),
    # Semiconductors Core
    "INTC": ("Intel", "Semiconductors", "IDM"),
    "TSM": ("Taiwan Semi", "Semiconductors", "Foundry"),
    "TXN": ("Texas Instruments", "Semiconductors", "Analog IDM"),
    "ADI": ("Analog Devices", "Semiconductors", "Analog IDM"),
    "AVGO": ("Broadcom", "Semiconductors", "Diversified/Fabless"),
    "QCOM": ("Qualcomm", "Semiconductors", "Wireless Fabless"),
    "NVDA": ("Nvidia", "Semiconductors", "Fabless (GPUs)"),
    "AMD": ("AMD", "Semiconductors", "Fabless (CPUs/GPUs)"),
    "MRVL": ("Marvell", "Semiconductors", "Fabless (Networking/Storage)"),
    "LSCC": ("Lattice Semi", "Semiconductors", "Fabless (FPGAs)"),
    "GFS": ("GlobalFoundries", "Semiconductors", "Foundry/IDM"),
    "ALGM": ("Allegro Micro", "Semiconductors", "Analog Auto/Industrial"),
    "CRDO": ("Credo Tech", "Semiconductors", "Fabless Networking"),
    "RMBS": ("Rambus", "Semiconductors", "Fabless IP/Memory"),
    # Electronic Design Automation (EDA)
    "SNPS": ("Synopsys", "Software (EDA)", "EDA"),
    "CDNS": ("Cadence", "Software (EDA)", "EDA"),
    "ANSS": ("Ansys", "Software (EDA)", "EDA")
}

def fetch_prices(tickers, years=3):
    """Download adjusted daily close prices for given tickers."""
    end = datetime.utcnow()
    start = end - timedelta(days=int(365.25 * years))
    df = yf.download(
        tickers, start=start, end=end, interval="1d",
        auto_adjust=True, progress=False
    )["Close"]
    if isinstance(df, pd.Series):
        df = df.to_frame()
    return df.sort_index().ffill()

def compute_stats_and_plots(rets, tickers, suffix):
    """Compute annualised vols + correlation matrices, save CSVs + plots."""
    rets_sel = rets[tickers].dropna(axis=1, how="any")
    if rets_sel.empty:
        return pd.Series(dtype=float), np.nan

    # Annualised vol
    ann_vol = rets_sel.std() * np.sqrt(TRADING_DAYS)
    ann_vol.name = "AnnVol"
    ann_vol.to_csv(OUTDIR / f"step1_ann_vol_{suffix}.csv", header=True)

    # --- Plot Top 15 ---
    top15 = ann_vol.sort_values(ascending=False).head(15)
    plt.figure(figsize=(10, 6))
    top15.sort_values().plot(kind="barh", color="steelblue")
    plt.xlabel("Annualised Volatility")
    plt.ylabel("Ticker")
    plt.title(f"Top 15 Annualised Volatilities ({suffix})")
    plt.tight_layout()
    plt.savefig(OUTDIR / f"step1_ann_vol_top15_{suffix}.png")
    plt.close()

    # --- Plot All ---
    sorted_vols = ann_vol.sort_values(ascending=False)
    plt.figure(figsize=(10, max(6, len(sorted_vols) * 0.35)))
    sorted_vols.sort_values().plot(kind="barh", color="steelblue")
    plt.xlabel("Annualised Volatility")
    plt.ylabel("Ticker")
    plt.title(f"All Annualised Volatilities ({suffix})")
    plt.tight_layout()
    plt.savefig(OUTDIR / f"step1_ann_vol_{suffix}.png")
    plt.close()

    # Correlation matrix + heatmap
    corr = rets_sel.corr()
    corr.to_csv(OUTDIR / f"step1_corr_{suffix}.csv")

    plt.figure(figsize=(10, 8))
    sns.heatmap(corr, cmap="coolwarm", center=0, annot=False, linewidths=0.3)
    plt.title(f"Correlation Matrix ({suffix})", fontsize=14)
    plt.xticks(rotation=90, fontsize=7)
    plt.yticks(fontsize=7)
    plt.tight_layout()
    plt.savefig(OUTDIR / f"step1_corr_{suffix}.png")
    plt.close()

    offdiag = corr.values[np.triu_indices_from(corr, k=1)]
    return ann_vol, np.median(offdiag)

def main():
    prices = fetch_prices(TICKERS, years=YEARS)

    # Keep only tickers with enough history
    day_counts = prices.count()
    keep = day_counts[day_counts >= MIN_DAYS].index.tolist()
    dropped = sorted(set(TICKERS) - set(keep))
    prices = prices[keep]

    # Returns
    rets = prices.pct_change().dropna()

    # Save all prices/returns
    prices.to_csv(OUTDIR / "step1_prices.csv")
    rets.to_csv(OUTDIR / "step1_returns_daily.csv")

    # Save dropped tickers
    if dropped:
        with open(OUTDIR / "step1_dropped_tickers.txt", "w") as f:
            f.write("\n".join(dropped))

    # Stats + plots: SEMIS (base universe)
    ann_vol_semis, medcorr_semis = compute_stats_and_plots(
        rets, [t for t in SEMIS if t in rets.columns], "semis"
    )

    # Stats + plots: ALL (semis + diversifiers + shorts only; exclude FACTOR_TICKERS)
    core_tickers = [t for t in SEMIS + DIVERSIFIERS + list(SHORTS.values()) if t in rets.columns]
    ann_vol_all, medcorr_all = compute_stats_and_plots(
        rets, core_tickers, "all"
    )

    # --- Universe classification table ---
    records = []
    for tkr in SEMIS:
        if tkr in CLASSIFICATION:
            name, gics, model = CLASSIFICATION[tkr]
            records.append({"Ticker": tkr, "Company": name, "GICS Sub-Industry": gics, "Business Model": model})
    df_class = pd.DataFrame(records)
    df_class.to_csv(OUTDIR / "step1_universe_table.csv", index=False)

    # Also save LaTeX for Overleaf
    with open(OUTDIR / "step1_universe_table.tex", "w") as f:
        f.write(df_class.to_latex(index=False, longtable=True, escape=False))

    # Console summary
    print(f"\n✓ Kept {len(keep)} tickers, dropped {len(dropped)} (<{MIN_DAYS} days)")
    if dropped:
        print("Dropped:", ", ".join(dropped))

    print("\nTop 15 by annualised vol (SEMIS, %):")
    print((ann_vol_semis.sort_values(ascending=False).head(15) * 100).round(2).astype(str) + "%")

    print(f"\nMedian off-diagonal correlation (SEMIS): {medcorr_semis:.2f}")
    print(f"Median off-diagonal correlation (ALL)  : {medcorr_all:.2f}")

    print("\nFiles saved:")
    print(" - step1_prices.csv, step1_returns_daily.csv")
    print(" - step1_ann_vol_semis.csv/.png, step1_ann_vol_top15_semis.png, step1_corr_semis.csv/.png")
    print(" - step1_ann_vol_all.csv/.png, step1_ann_vol_top15_all.png, step1_corr_all.csv/.png")
    print(" - step1_dropped_tickers.txt")
    print(" - step1_universe_table.csv/.tex")
    print(" - (FACTOR_TICKERS downloaded but excluded from stats/plots)")

if __name__ == "__main__":
    main()



✓ Kept 39 tickers, dropped 0 (<500 days)

Top 15 by annualised vol (SEMIS, %):
Ticker
CRDO    79.92%
ICHR    61.56%
MRVL    60.62%
LSCC    54.47%
NVDA    53.19%
RMBS    53.02%
ENTG    51.83%
ALGM    51.24%
AMD     51.14%
INTC    49.58%
AVGO    45.89%
TER     44.54%
LRCX    43.96%
GFS     43.93%
COHU    42.19%
Name: AnnVol, dtype: object

Median off-diagonal correlation (SEMIS): 0.59
Median off-diagonal correlation (ALL)  : 0.42

Files saved:
 - step1_prices.csv, step1_returns_daily.csv
 - step1_ann_vol_semis.csv/.png, step1_ann_vol_top15_semis.png, step1_corr_semis.csv/.png
 - step1_ann_vol_all.csv/.png, step1_ann_vol_top15_all.png, step1_corr_all.csv/.png
 - step1_dropped_tickers.txt
 - step1_universe_table.csv/.tex
 - (FACTOR_TICKERS downloaded but excluded from stats/plots)


In [29]:
# pull in t-bills and 2.compute_excess_returns.py
from pathlib import Path
import pandas as pd
import numpy as np
from pandas_datareader.data import DataReader

IN_RET = Path("outputs/step1_returns_daily.csv")
OUTDIR = Path("outputs"); OUTDIR.mkdir(parents=True, exist_ok=True)

def fetch_rf_daily(start, end):
    # 3M Treasury Constant Maturity (daily, %)
    rf = DataReader("DGS3MO", "fred", start, end)
    rf = rf.rename(columns={"DGS3MO": "DGS3MO_%"}).sort_index()
    # Convert % annual -> daily decimal (approx 252 trading days)
    rf["rf_daily"] = (rf["DGS3MO_%"] / 100.0) / 252.0
    # Forward-fill to trading days later
    return rf[["rf_daily"]]

def main():
    # Load daily simple returns from Step 1
    rets = pd.read_csv(IN_RET, index_col=0, parse_dates=True)
    rets = rets.sort_index()

    # Fetch risk-free over same window
    rf = fetch_rf_daily(rets.index.min(), rets.index.max())

    # Align to trading days and forward-fill gaps (weekends/holidays)
    rf_aligned = rf.reindex(rets.index).ffill().fillna(0.0)["rf_daily"]

    # Excess returns = asset returns - rf_daily (per column)
    excess = rets.sub(rf_aligned, axis=0)

    # --- Sanity checks to prevent silent drift ---
    assert rets.index.equals(excess.index), "Index mismatch: returns vs excess returns"
    assert rets.index.equals(rf_aligned.index), "Index mismatch: returns vs risk-free series"
    assert excess.notna().all().all(), "NaNs found in excess returns!"

    # Save
    rf_aligned.to_frame(name="rf_daily").to_csv(OUTDIR / "step2_rf_daily.csv")
    excess.to_csv(OUTDIR / "step2_excess_returns.csv")

    # Console summary
    rf_ann = rf_aligned.mean() * 252
    print(f"✓ Saved risk-free series → {OUTDIR/'step2_rf_daily.csv'}")
    print(f"✓ Saved excess returns   → {OUTDIR/'step2_excess_returns.csv'}")
    print(f"Avg annualized RF over sample: {rf_ann:.2%}")
    print("Excess returns preview:\n", excess.head())

if __name__ == "__main__":
    main()


✓ Saved risk-free series → outputs/step2_rf_daily.csv
✓ Saved excess returns   → outputs/step2_excess_returns.csv
Avg annualized RF over sample: 4.88%
Excess returns preview:
                  ADI      ALGM      AMAT       AMD      AMZN      ANSS  \
Date                                                                     
2022-08-23  0.003368  0.001917  0.010999 -0.003881  0.002891 -0.013051   
2022-08-24 -0.004735  0.010819 -0.001981  0.002483  0.001235  0.004129   
2022-08-25  0.020784  0.024315  0.034573  0.047874  0.025895  0.025801   
2022-08-26 -0.050393 -0.045854 -0.059162 -0.061856 -0.047682 -0.048391   
2022-08-29 -0.017343 -0.025927 -0.022992 -0.029620 -0.007460 -0.012504   

                ASML      AVGO      CDNS      COHU  ...       SMH      SNPS  \
Date                                                ...                       
2022-08-23  0.010810  0.001837 -0.009035  0.007724  ...  0.006370  0.002402   
2022-08-24 -0.000701  0.001889 -0.001531  0.013669  ... -0.000769 -0

In [30]:
# step3_covariances.py  (safe Overleaf table patch)

from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.covariance import LedoitWolf

OUTDIR = Path("outputs")
OUTDIR.mkdir(exist_ok=True, parents=True)

# --- Asset Universes (core groups stay fixed) ---
SEMIS = [
    "NVDA","AMD","AVGO","QCOM","TSM","ASML","LRCX","TXN","INTC","ADI",
    "MRVL","KLAC","LSCC","GFS","AMAT","ALGM","CRDO","RMBS",
    "TER","COHU","ENTG","ICHR","SNPS","CDNS","ANSS"
]
DIVERSIFIERS = ["TLT","GLD","CPER","USO","XBI","VIXY","LQD"]

# Shorts: synthetic tickers we invert
SHORTS = {"SHORT_CPER": "CPER", "SHORT_USO": "USO", "SHORT_GLD": "GLD"}

# Report-specific settings (only affect the Overleaf .tex table)
REPORT_DIVERSIFIERS = ["GLD", "TLT", "VIXY"]   # only these 3 in the paper
REPORT_SORT_BY = "vol"  # "vol" for volatility (desc) or "drawdown" for worst-first

def detect_factors(all_tickers):
    known = set(SEMIS + DIVERSIFIERS + list(SHORTS.keys()))
    return [t for t in all_tickers if t not in known]

def diag(mat: pd.DataFrame):
    eig = np.linalg.eigvalsh(mat.values)
    lam_min, lam_max = float(eig.min()), float(eig.max())
    cond = (lam_max / lam_min) if lam_min > 0 else np.inf
    return lam_min, lam_max, cond

def top_bottom_corr(corr: pd.DataFrame, n=10):
    corr_long = (
        corr.stack()
        .reset_index()
        .rename(columns={0: "Correlation", "level_0": "Asset1", "level_1": "Asset2"})
    )
    corr_long = corr_long[corr_long["Asset1"] < corr_long["Asset2"]]
    return (
        corr_long.nsmallest(n, "Correlation").reset_index(drop=True),
        corr_long.nlargest(n, "Correlation").reset_index(drop=True),
    )

def compute_drawdowns(sub: pd.DataFrame):
    dd = {}
    for col in sub:
        cum = (1 + sub[col]).cumprod()
        running_max = cum.cummax()
        drawdown = (cum / running_max - 1.0).min()
        dd[col] = float(drawdown)
    return pd.Series(dd, name="max_drawdown")

def process_subset(rets: pd.DataFrame, tickers: list, suffix: str):
    sub = rets[[t for t in tickers if t in rets.columns]].dropna(axis=1, how="any")
    if sub.empty:
        print(f"⚠️ No valid returns for {suffix}")
        return

    # Covariance & shrinkage (annualised)
    S_sample_ann = sub.cov() * 252.0
    lw = LedoitWolf().fit(sub.values)
    S_lw_ann = pd.DataFrame(lw.covariance_, index=sub.columns, columns=sub.columns) * 252.0

    # Correlation
    corr = sub.corr()

    # Drawdowns & vol
    dd = compute_drawdowns(sub)
    vols = sub.std() * np.sqrt(252)
    vol_dd = pd.concat([vols.rename("Annualised Volatility"), dd], axis=1)
    vol_dd.to_latex(OUTDIR / f"step3_vol_dd_{suffix}.tex", float_format="%.2f")

    # Heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr, cmap="coolwarm", center=0, annot=False, linewidths=0.3)
    plt.title(f"Correlation Matrix (Excess, {suffix})", fontsize=14)
    plt.tight_layout()
    plt.savefig(OUTDIR / f"step3_corr_{suffix}.png")
    plt.close()

    # Top/Bottom correlation pairs
    bottom, top = top_bottom_corr(corr, n=10)
    bottom.to_latex(OUTDIR / f"step3_lowest_corr_pairs_{suffix}.tex", index=False, float_format="%.2f")
    top.to_latex(OUTDIR / f"step3_highest_corr_pairs_{suffix}.tex", index=False, float_format="%.2f")

    print(f"\nSubset: {suffix} → wrote .tex + .png to outputs/")

def main():
    rets = pd.read_csv(OUTDIR/"step2_excess_returns.csv", index_col=0, parse_dates=True).sort_index()

    # Invert SHORT tickers
    for short_name, underlying in SHORTS.items():
        if underlying in rets.columns:
            rets[short_name] = -rets[underlying]

    # Drop NaNs
    rets = rets.dropna(how="any")

    # Factor detection
    FACTORS = detect_factors(rets.columns.tolist())

    # --- Original analytics (unchanged) ---
    process_subset(rets, [t for t in SEMIS if t in rets.columns], "semis")
    process_subset(rets, [t for t in DIVERSIFIERS + list(SHORTS.keys()) if t in rets.columns], "diversifiers")
    if FACTORS:
        process_subset(rets, FACTORS, "factors")
    process_subset(rets, rets.columns.tolist(), "all")  # keep original 'all' outputs

    # --- Overleaf-specific curated table (does NOT change analytics) ---
    # 1) Backup the original 'all' vol/DD table as _full.tex for traceability
    try:
        # Recompute quickly rather than parsing LaTeX
        sub_full = rets[rets.columns].dropna(axis=1, how="any")
        vols_full = sub_full.std() * np.sqrt(252)
        dd_full = compute_drawdowns(sub_full)
        vol_dd_full = pd.concat([vols_full.rename("Annualised Volatility"), dd_full], axis=1)
        vol_dd_full.to_latex(OUTDIR / "step3_vol_dd_all_full.tex", float_format="%.2f")
    except Exception as e:
        print(f"⚠️ Could not write step3_vol_dd_all_full.tex backup: {e}")

    # 2) Write curated Semis + GLD/TLT/VIXY to step3_vol_dd_all.tex (the file Overleaf includes)
    report_cols = [t for t in (SEMIS + REPORT_DIVERSIFIERS) if t in rets.columns]
    sub_rep = rets[report_cols].dropna(axis=1, how="any")
    if not sub_rep.empty:
        vols_rep = sub_rep.std() * np.sqrt(252)
        dd_rep = compute_drawdowns(sub_rep)
        vol_dd_rep = pd.concat([vols_rep.rename("Annualised Volatility"), dd_rep], axis=1)

        if REPORT_SORT_BY.lower() == "vol":
            vol_dd_rep = vol_dd_rep.sort_values(by="Annualised Volatility", ascending=False)
        elif REPORT_SORT_BY.lower() == "drawdown":
            vol_dd_rep = vol_dd_rep.sort_values(by="max_drawdown", ascending=True)

        vol_dd_rep.to_latex(OUTDIR / "step3_vol_dd_all.tex", float_format="%.2f")
        print("Overleaf table written: outputs/step3_vol_dd_all.tex (Semis + GLD/TLT/VIXY)")
    else:
        print("⚠️ Curated Overleaf table empty (no overlapping columns).")

if __name__ == "__main__":
    main()



Subset: semis → wrote .tex + .png to outputs/

Subset: diversifiers → wrote .tex + .png to outputs/

Subset: factors → wrote .tex + .png to outputs/

Subset: all → wrote .tex + .png to outputs/
Overleaf table written: outputs/step3_vol_dd_all.tex (Semis + GLD/TLT/VIXY)


In [31]:
# step4_factors.py

from pathlib import Path
import pandas as pd
import numpy as np
import pandas_datareader.data as web

OUTDIR = Path("outputs"); OUTDIR.mkdir(parents=True, exist_ok=True)
EXCESS_FILE = OUTDIR / "step2_excess_returns.csv"
FACTORS_FILE = OUTDIR / "step4_all_factors.csv"

# -------------------------
# Helpers
# -------------------------
def safe_mean(df, cols, name):
    """Return equal-weight basket return if all columns exist, else NaN series."""
    missing = [c for c in cols if c not in df.columns]
    if missing:
        print(f"⚠️ Skipping {name}, missing: {missing}")
        return pd.Series(np.nan, index=df.index, name=name)
    return df[cols].mean(axis=1).rename(name)

# -------------------------
# Main
# -------------------------
def main():
    # Load excess returns
    excess = pd.read_csv(EXCESS_FILE, index_col=0, parse_dates=True)
    excess = excess.apply(pd.to_numeric, errors="coerce")

    # -------------------------
    # Custom baskets
    # -------------------------
    semi = safe_mean(excess, ["NVDA", "AMD", "INTC"], "SemiBasket")

    hypers = safe_mean(excess, ["MSFT", "AMZN", "GOOGL"], "HyperscalerBasket")

    semi_capex = safe_mean(excess, ["AMAT", "ASML", "LRCX", "KLAC"], "SemiCapex")

    # hyperscaler capex proxy = same hyperscalers (in practice: infra spend)
    hyper_capex = safe_mean(excess, ["MSFT", "AMZN", "GOOGL"], "HyperCapex")

    semi_vs_hypers = (semi - hypers).rename("Semi_vs_Hypers")
    semi_capex_vs_hypercapex = (semi_capex - hyper_capex).rename("SemiCapex_vs_HyperCapex")

    custom_factors = pd.concat(
        [semi, hypers, semi_capex, hyper_capex, semi_vs_hypers, semi_capex_vs_hypercapex],
        axis=1
    )

    # -------------------------
    # Fama-French Factors
    # -------------------------
    try:
        ff5 = web.DataReader("F-F_Research_Data_5_Factors_2x3_daily", "famafrench")[0]
        ff5.index = pd.to_datetime(ff5.index, format="%Y%m%d")
        ff5 = ff5.rename_axis("Date") / 100.0  # convert % to decimals
    except Exception as e:
        print(f"⚠️ Could not download Fama-French 5-factors: {e}")
        ff5 = pd.DataFrame(index=excess.index)

    try:
        mom = web.DataReader("F-F_Momentum_Factor_daily", "famafrench")[0]
        mom.index = pd.to_datetime(mom.index, format="%Y%m%d")
        mom = mom.rename(columns={"Mom   ": "Mom"}) / 100.0
    except Exception as e:
        print(f"⚠️ Could not download Momentum factor: {e}")
        mom = pd.DataFrame(index=excess.index)

    # -------------------------
    # Combine everything
    # -------------------------
    all_factors = pd.concat([custom_factors, ff5, mom], axis=1)
    all_factors = all_factors.loc[excess.index].sort_index()
    all_factors = all_factors.apply(pd.to_numeric, errors="coerce")

    all_factors.to_csv(FACTORS_FILE)
    print(f"✅ Saved factors to {FACTORS_FILE}")
    print("Columns:", list(all_factors.columns))

# -------------------------
if __name__ == "__main__":
    main()


✅ Saved factors to outputs/step4_all_factors.csv
Columns: ['SemiBasket', 'HyperscalerBasket', 'SemiCapex', 'HyperCapex', 'Semi_vs_Hypers', 'SemiCapex_vs_HyperCapex', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'Mom']


In [32]:
# step5_backtest_factors.py (fully fixed)

from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import statsmodels.api as sm

OUTDIR = Path("outputs"); OUTDIR.mkdir(parents=True, exist_ok=True)

EXCESS_FILE = OUTDIR / "step2_excess_returns.csv"
FACTORS_FILE = OUTDIR / "step4_all_factors.csv"

# -------------------------
# Helpers
# -------------------------
def compute_perf(returns, freq=252):
    """Compute performance statistics for a return series."""
    returns = returns.dropna()
    if returns.empty:
        return {"AnnRet": np.nan, "AnnVol": np.nan, "Sharpe": np.nan, "MaxDD": np.nan}
    mu = returns.mean() * freq
    vol = returns.std() * np.sqrt(freq)
    sharpe = mu / vol if vol > 0 else np.nan
    cum = (1 + returns).cumprod()
    mdd = (cum / cum.cummax() - 1).min()
    return {"AnnRet": mu, "AnnVol": vol, "Sharpe": sharpe, "MaxDD": mdd}

def beautify_plot(ax):
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b-%y'))
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right")
    ax.grid(True, ls="--", alpha=0.6)
    return ax

def add_line_labels(ax, spacing=5):
    """Put labels at end of each line, but skip if last point is NaN."""
    for line in ax.get_lines():
        xdata = pd.Series(line.get_xdata())
        ydata = pd.Series(line.get_ydata())
        label = line.get_label()
        mask = ydata.dropna()
        if mask.empty:
            continue
        x_last, y_last = xdata.iloc[mask.index[-1]], ydata.iloc[mask.index[-1]]
        if not np.isfinite(y_last):
            continue
        ax.text(
            x_last + pd.Timedelta(days=spacing),
            y_last,
            label,
            color=line.get_color(),
            va="center",
            fontsize=8
        )

# -------------------------
# Rolling regression
# -------------------------
def rolling_regression(y, x, window=252, min_obs=50):
    betas, alphas, dates = [], [], []
    for i in range(window, len(y)):
        y_win = y.iloc[i-window:i]
        x_win = x.iloc[i-window:i]
        df = pd.concat([y_win, x_win], axis=1).dropna()
        if len(df) < min_obs:
            betas.append(np.nan); alphas.append(np.nan); dates.append(y.index[i]); continue
        y_clean = df.iloc[:, 0]
        X_clean = sm.add_constant(df.iloc[:, 1])
        try:
            res = sm.OLS(y_clean, X_clean, missing="drop").fit()
            betas.append(res.params.iloc[1])
            alphas.append(res.params.iloc[0])
        except Exception:
            betas.append(np.nan); alphas.append(np.nan)
        dates.append(y.index[i])
    return pd.Series(betas, index=dates), pd.Series(alphas, index=dates)

# -------------------------
# Main
# -------------------------
def main():
    excess = pd.read_csv(EXCESS_FILE, index_col=0, parse_dates=True).sort_index()
    factors = pd.read_csv(FACTORS_FILE, index_col=0, parse_dates=True).sort_index()

    # Standardize factor column names
    factors.columns = factors.columns.str.strip()
    if "Mkt-RF" not in factors.columns:
        for col in factors.columns:
            if col.lower().replace("-", "").replace("_", "") in ["mktrf", "marketexcess", "mkt"]:
                factors = factors.rename(columns={col: "Mkt-RF"})

    # --- Truncate FF factors to 30th June of last available year ---
    cutoff = pd.Timestamp(f"{factors.index[-1].year}-06-30")
    ff_cols = ["Mkt-RF", "SMB", "HML", "RMW", "CMA", "RF", "Mom"]
    for col in [c for c in ff_cols if c in factors.columns]:
        factors.loc[factors.index > cutoff, col] = np.nan

    # Align with excess returns
    idx = excess.index.intersection(factors.index)
    factors, excess = factors.loc[idx], excess.loc[idx]

    # 1. Factor performance
    perf_stats = {col: compute_perf(factors[col]) for col in factors.columns}
    perf_df = pd.DataFrame(perf_stats).T
    perf_df.to_csv(OUTDIR / "step5_factor_performance.csv")

    # Combined cumulative returns (skip leading NaNs, no fillna(0))
    cumrets = (1 + factors).cumprod() * 10000
    ax = cumrets.plot(figsize=(10, 6), lw=1)
    ax.set_title("Cumulative Returns of Factors")
    ax.set_ylabel("Growth of $10,000")
    ax.set_xlabel("Date")
    beautify_plot(ax)
    add_line_labels(ax)
    ax.legend().remove()
    plt.tight_layout()
    plt.savefig(OUTDIR / "step5_cumulative_returns.png", dpi=150)
    plt.close()

    # Individual cumulative plots
    for fac in factors.columns:
        series = factors[fac].copy()
        series = series.loc[series.first_valid_index():]  # drop leading NaN
        cumret = (1 + series).cumprod() * 10000

        ax = cumret.plot(figsize=(8, 4), lw=1)
        ax.set_title(f"Cumulative Return: {fac}")
        ax.set_ylabel("Growth of $10,000")
        ax.set_xlabel("Date")
        beautify_plot(ax)
        plt.tight_layout()
        plt.savefig(OUTDIR / f"step5_cumulative_{fac}.png", dpi=150)
        plt.close()

    # -------------------------
    # Rolling regressions: all assets vs each factor
    # -------------------------
    window = 252
    all_betas, all_alphas = {}, {}

    for asset in excess.columns:
        y = excess[asset]
        for fac in [f for f in factors.columns if f != "RF"]:
            beta, alpha = rolling_regression(y, factors[[fac]], window=window)
            all_betas[(asset, fac)] = beta
            all_alphas[(asset, fac)] = alpha

            # Plot betas
            if not beta.dropna().empty:
                ax = beta.plot(figsize=(8,4), lw=1)
                ax.set_title(f"Rolling {window}d Beta: {asset} vs {fac}")
                ax.set_ylabel("Beta")
                beautify_plot(ax)
                plt.tight_layout()
                plt.savefig(OUTDIR / f"step5_beta_{asset}_{fac}.png", dpi=150)
                plt.close()

            # Plot alphas
            if not alpha.dropna().empty:
                ax = alpha.plot(figsize=(8,4), lw=1, color="purple")
                ax.set_title(f"Rolling {window}d Alpha: {asset} vs {fac}")
                ax.set_ylabel("Alpha")
                beautify_plot(ax)
                plt.tight_layout()
                plt.savefig(OUTDIR / f"step5_alpha_{asset}_{fac}.png", dpi=150)
                plt.close()

    # Save rolling betas/alphas to wide-format CSVs
    betas_df = pd.DataFrame(all_betas)
    alphas_df = pd.DataFrame(all_alphas)
    betas_df.to_csv(OUTDIR / "step5_rolling_betas.csv")
    alphas_df.to_csv(OUTDIR / "step5_rolling_alphas.csv")

    # Factor summary table (no per-asset betas here, just factor stats)
    final_growth = (1 + factors).cumprod() * 10000
    summary = pd.DataFrame({
        "FinalValue($10k)": final_growth.iloc[-1].round(2),
        "AnnRet": perf_df["AnnRet"],
        "Sharpe": perf_df["Sharpe"]
    })
    summary.to_csv(OUTDIR / "step5_factor_summary.csv")
    summary.round(3).to_latex(OUTDIR / "step5_factor_summary.tex")

    print("\n✓ Step 5 complete: factor performance, cumulative plots, rolling regressions (all assets), summary table.")

if __name__ == "__main__":
    main()



✓ Step 5 complete: factor performance, cumulative plots, rolling regressions (all assets), summary table.


In [40]:
# step5c_backtest_portfolio.py

from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import statsmodels.api as sm

OUTDIR = Path("outputs"); OUTDIR.mkdir(parents=True, exist_ok=True)

EXCESS_FILE = OUTDIR / "step2_excess_returns.csv"
FACTORS_FILE = OUTDIR / "step4_all_factors.csv"

# ---- Config ----
SEMIS = ["NVDA","AMD","ASML","AMAT","TSM","LRCX","INTC"]
DIVERSIFIERS = ["GLD","TLT","VIXY"]
NAMES = SEMIS + DIVERSIFIERS
BENCHMARK = "SMH"

# ---- Hard-coded Market Caps (USD) ----
MKT_CAPS = {
    "ASML": 295.706e9,
    "TSM": 1023.605e9,
    "AMAT": 131.610e9,
    "AMD": 282.294e9,
    "INTC": 109.556e9,
    "LRCX": 125.309e9,
    "NVDA": 4449.584e9,
    # scaled for diversifiers for meaningful presence
    "GLD": 250e9,
    "TLT": 500e9,
    "VIXY": 50e9,
}

# -------------------------
# Helpers
# -------------------------
def compute_perf(returns, freq=252):
    """Performance metrics for a return series."""
    r = returns.dropna()
    if r.empty:
        return {"AnnRet": np.nan, "AnnVol": np.nan, "Sharpe": np.nan, "MaxDD": np.nan}
    mu = r.mean() * freq
    vol = r.std() * np.sqrt(freq)
    sharpe = mu / vol if vol > 0 else np.nan
    cum = (1 + r).cumprod()
    mdd = (cum / cum.cummax() - 1).min()
    return {"AnnRet": mu, "AnnVol": vol, "Sharpe": sharpe, "MaxDD": mdd}

def beautify(ax):
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%b-%y"))
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right")
    ax.grid(True, ls="--", alpha=0.6)
    return ax

def ols_hac_table(y, X, nw_lags=20):
    """OLS with HAC (Newey–West) SEs; returns a tidy table."""
    df = pd.concat([y, X], axis=1).dropna()
    if df.empty or df.shape[1] < 2:
        return pd.DataFrame()
    y_clean = df.iloc[:, 0]
    X_clean = sm.add_constant(df.iloc[:, 1:])
    res = sm.OLS(y_clean, X_clean).fit(cov_type="HAC", cov_kwds={"maxlags": nw_lags})
    out = pd.DataFrame({"coef": res.params, "tstat": res.tvalues, "pval": res.pvalues})
    out.loc["R2", ["coef","tstat","pval"]] = [res.rsquared, np.nan, np.nan]
    # pretty p-values for LaTeX
    def fmt_p(v):
        try:
            if np.isfinite(v):
                return "<0.001" if v < 1e-3 else f"{v:.3f}"
        except Exception:
            pass
        return ""
    out["pval"] = out["pval"].apply(fmt_p)
    return out

def rolling_regression(y, x, window=252, min_obs=50):
    """Rolling regression of y on a single factor x (DataFrame with one column)."""
    betas, alphas, dates = [], [], []
    for i in range(window, len(y)):
        df = pd.concat([y.iloc[i-window:i], x.iloc[i-window:i]], axis=1).dropna()
        dates.append(y.index[i])
        if len(df) < min_obs:
            betas.append(np.nan); alphas.append(np.nan); continue
        res = sm.OLS(df.iloc[:,0], sm.add_constant(df.iloc[:,1])).fit()
        betas.append(res.params.iloc[1])
        alphas.append(res.params.iloc[0])
    return pd.Series(betas, index=dates), pd.Series(alphas, index=dates)

# -------------------------
# Portfolio Constructors
# -------------------------
def build_equal_weight(excess):
    names_present = [c for c in NAMES if c in excess.columns]
    missing = sorted(set(NAMES) - set(names_present))
    if missing:
        print(f"⚠️  Missing in returns (EW): {missing}")
    if not names_present:
        raise ValueError("No portfolio names found in excess returns.")
    return excess[names_present].mean(axis=1).rename("Portfolio_EW")

def build_mcap_weight(excess):
    names_present = [c for c in NAMES if c in excess.columns]
    missing = sorted(set(NAMES) - set(names_present))
    if missing:
        print(f"⚠️  Missing in returns (MCAP): {missing}")

    weights = pd.Series(MKT_CAPS).reindex(names_present).dropna()
    weights = weights / weights.sum()

    print("\n📊 Hard-coded Market-cap weights (normalised):")
    print(weights.round(3))

    # Save CSV and LaTeX for Overleaf
    OUTDIR.mkdir(parents=True, exist_ok=True)
    weights.to_csv(OUTDIR / "step5_mcap_weights.csv")

    wdf = weights.rename("Weight").to_frame()
    wdf.index.name = "Ticker"
    wdf.loc["Total", "Weight"] = wdf["Weight"].sum()
    wdf.to_latex(OUTDIR / "step5_mcap_weights.tex",
                 float_format=lambda x: f"{x:.3f}",
                 index=True, escape=False)

    return (excess[names_present] @ weights).rename("Portfolio_MCAP")

# -------------------------
# Main
# -------------------------
def main():
    # Load data
    excess = pd.read_csv(EXCESS_FILE, index_col=0, parse_dates=True).sort_index()
    factors = pd.read_csv(FACTORS_FILE, index_col=0, parse_dates=True).sort_index()

    # Align dates
    idx = excess.index.intersection(factors.index)
    excess, factors = excess.loc[idx], factors.loc[idx]

    # Benchmark
    if BENCHMARK not in excess.columns:
        raise KeyError(f"Benchmark {BENCHMARK} not found in excess returns columns.")
    bench = excess[BENCHMARK].rename("Benchmark")

    # Build portfolios
    ew = build_equal_weight(excess)
    mcap = build_mcap_weight(excess)
    portfolios = pd.concat([ew, mcap, bench], axis=1)

    # --- Performance comparison ---
    perf = pd.DataFrame({col: compute_perf(portfolios[col]) for col in portfolios.columns}).T
    perf.to_latex(OUTDIR / "step5_perf_portfolios_vs_smh.tex", float_format="%.3f", escape=True)

    # --- Cumulative returns plot ---
    cum = (1 + portfolios).cumprod() * 10000
    ax = cum.plot(figsize=(9, 6))
    ax.set_title("Portfolio (EW & MCAP) vs SMH")
    ax.set_ylabel("Growth of $10,000")
    beautify(ax)
    plt.tight_layout()
    plt.savefig(OUTDIR / "step5_portfolios_vs_smh.png", dpi=150)
    plt.close()

    # --- Static regressions (HAC / Newey–West) ---
    # Normalise factor column names (handle both Mkt_RF and Mkt-RF)
    factors.columns = [c.strip().replace("Mkt_RF", "Mkt-RF") for c in factors.columns]

    NW_LAGS = 20  # ~1 trading month (tweakable)
    # Two specs: main "levels" for paper; "spread" optional for appendix
    LEVELS = ["Mkt-RF","SMB","HML","RMW","CMA","Mom","SemiCapex"]
    SPREAD = ["Mkt-RF","SMB","HML","RMW","CMA","Mom","Semi vs Hypers"]

    def safe_pick(cols):
        return [c for c in cols if c in factors.columns]

    for col in ["Portfolio_EW", "Portfolio_MCAP"]:
        # MAIN tables (levels) to canonical filenames used by your LaTeX
        X_levels = factors.reindex(columns=safe_pick(LEVELS))
        tbl_levels = ols_hac_table(portfolios[col], X_levels, nw_lags=NW_LAGS)
        if not tbl_levels.empty:
            tbl_levels.to_latex(OUTDIR / f"step5_regressions_{col}.tex",
                                float_format="%.3f", escape=False)

        # APPENDIX tables (spread) to side files (optional to include)
        X_spread = factors.reindex(columns=safe_pick(SPREAD))
        tbl_spread = ols_hac_table(portfolios[col], X_spread, nw_lags=NW_LAGS)
        if not tbl_spread.empty:
            tbl_spread.to_latex(OUTDIR / f"step5_regressions_{col}_spread.tex",
                                float_format="%.3f", escape=False)

    # --- Rolling regressions ---
    fac_list = [c for c in factors.columns if c != "RF"]
    for col in ["Portfolio_EW", "Portfolio_MCAP"]:
        for fac in fac_list:
            beta, alpha = rolling_regression(portfolios[col], factors[[fac]])
            if not beta.dropna().empty:
                ax = beta.plot(figsize=(8, 4))
                ax.set_title(f"Rolling Beta: {col} vs {fac}")
                ax.set_ylabel("Beta")
                beautify(ax)
                plt.tight_layout()
                plt.savefig(OUTDIR / f"step5_rolling_beta_{col}_{fac}.png", dpi=150)
                plt.close()
            if not alpha.dropna().empty:
                ax = alpha.plot(figsize=(8, 4))
                ax.set_title(f"Rolling Alpha: {col} vs {fac}")
                ax.set_ylabel("Alpha")
                beautify(ax)
                plt.tight_layout()
                plt.savefig(OUTDIR / f"step5_rolling_alpha_{col}_{fac}.png", dpi=150)
                plt.close()

    print("✓ Step 5c complete: EW & MCAP portfolios vs SMH, HAC regressions, rolling betas/alphas, and LaTeX tables/images written.")

if __name__ == "__main__":
    main()



📊 Hard-coded Market-cap weights (normalised):
NVDA    0.616
AMD     0.039
ASML    0.041
AMAT    0.018
TSM     0.142
LRCX    0.017
INTC    0.015
GLD     0.035
TLT     0.069
VIXY    0.007
dtype: float64
✓ Step 5c complete: EW & MCAP portfolios vs SMH, HAC regressions, rolling betas/alphas, and LaTeX tables/images written.


In [4]:
# step5d_risk_corr.py (diagnostics: block medians, ex-VIXY check, long-form pairs, data QA)

from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

OUTDIR = Path("outputs"); OUTDIR.mkdir(parents=True, exist_ok=True)

EXCESS_FILE = OUTDIR / "step2_excess_returns.csv"

# ---- Config ----
SEMIS = ["NVDA","AMD","ASML","AMAT","TSM","LRCX","INTC"]
DIVERSIFIERS = ["GLD","TLT","VIXY"]
BASE = SEMIS
ALL = SEMIS + DIVERSIFIERS
BENCHMARK = "SMH"   # display only

# Reference weights (MCAP-style) for risk contributions (on ALL)
FINAL_WEIGHTS = {
    "ASML": 0.041,
    "TSM":  0.142,
    "AMAT": 0.018,
    "AMD":  0.039,
    "INTC": 0.015,
    "LRCX": 0.017,
    "NVDA": 0.616,
    "GLD":  0.035,
    "TLT":  0.069,
    "VIXY": 0.007,
}

# -------------------------
# Helpers
# -------------------------
def ensure_cols(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    present = [c for c in cols if c in df.columns]
    missing = [c for c in cols if c not in df.columns]
    if missing:
        print(f"⚠️ Missing columns skipped: {missing}")
    return df[present]

def ann_vol(series: pd.Series, freq: int = 252) -> float:
    return float(series.std() * np.sqrt(freq))

def offdiag_median_corr(df: pd.DataFrame) -> float:
    C = df.corr()
    n = C.shape[0]
    if n <= 1:
        return np.nan
    mask = ~np.eye(n, dtype=bool)
    return float(C.where(mask).stack().median())

def save_corr_heatmap(df: pd.DataFrame, fname_png: Path, title: str):
    corr = df.corr()
    fig, ax = plt.subplots(figsize=(8, 6))
    im = ax.imshow(corr.values, cmap="coolwarm", vmin=-1, vmax=1)
    ax.set_xticks(range(len(corr.columns)))
    ax.set_yticks(range(len(corr.columns)))
    ax.set_xticklabels(corr.columns, rotation=45, ha="right")
    ax.set_yticklabels(corr.columns)
    fig.colorbar(im, ax=ax)
    ax.set_title(title)
    plt.tight_layout()
    plt.savefig(fname_png, dpi=150)
    plt.close()
    return corr

def risk_contrib_table(returns: pd.DataFrame, weights: dict[str, float]) -> pd.DataFrame:
    cov = returns.cov() * 252.0
    w = pd.Series(weights, dtype=float).reindex(cov.columns).fillna(0.0)
    sigma_p = float(np.sqrt(np.dot(w, np.dot(cov.values, w))))
    if sigma_p == 0:
        mrc = pd.Series(0.0, index=cov.columns)
        prc = pd.Series(0.0, index=cov.columns)
    else:
        mrc = pd.Series(np.dot(cov.values, w) / sigma_p, index=cov.columns)
        prc = (w * mrc) / sigma_p
    out = pd.DataFrame({"Weight": w, "MRC": mrc, "PRC": prc}, index=cov.columns)
    out = out.loc[out["Weight"].sort_values(ascending=False).index]
    return out

def write_vols_table(returns: pd.DataFrame, fname_tex: Path, median_offdiag: float):
    vols = returns.apply(ann_vol, axis=0).rename("AnnVol").to_frame()
    vols.index.name = "Ticker"
    summary_row = pd.DataFrame({"AnnVol": [median_offdiag]}, index=["Median off-diag corr"])
    table = pd.concat([vols, summary_row])
    table.to_latex(fname_tex, float_format="%.3f", escape=True)

def pairwise_longform(df: pd.DataFrame) -> pd.DataFrame:
    C = df.corr()
    tri = C.where(~np.tril(np.ones(C.shape, dtype=bool)))  # keep upper triangle off-diag
    long = tri.stack().reset_index()
    long.columns = ["A", "B", "corr"]
    return long

def block_medians(long_pairs: pd.DataFrame, semis: set, divers: set) -> dict:
    def tag(a, b):
        if a in semis and b in semis:
            return "semi-semi"
        if (a in semis and b in divers) or (a in divers and b in semis):
            return "semi-div"
        if a in divers and b in divers:
            return "div-div"
        return "other"
    t = long_pairs.copy()
    t["block"] = [tag(a, b) for a, b in zip(t["A"], t["B"])]
    out = {}
    for blk in ["semi-semi", "semi-div", "div-div", "other"]:
        s = t.loc[t["block"] == blk, "corr"]
        if not s.empty:
            out[blk] = float(s.median())
    out["n_pairs_total"] = int(len(t))
    out["n_pairs_by_block"] = {blk: int((t["block"] == blk).sum()) for blk in ["semi-semi","semi-div","div-div","other"]}
    return out

def data_quality_checks(df: pd.DataFrame, name: str):
    # duplicate dates
    if df.index.duplicated().any():
        dup_n = int(df.index.duplicated().sum())
        print(f"⚠️ {name}: dropped {dup_n} duplicated dates.")
        df = df[~df.index.duplicated(keep="first")]
    # constant/near-constant series
    sds = df.std()
    const_cols = sds.index[sds < 1e-12].tolist()
    if const_cols:
        print(f"⚠️ {name}: near-constant series detected (std<1e-12): {const_cols}")
    return df

# -------------------------
# Main
# -------------------------
def main():
    # Load returns (fresh each run)
    excess = pd.read_csv(EXCESS_FILE, index_col=0, parse_dates=True).sort_index()
    excess = data_quality_checks(excess, "excess")

    # Build aligned window on ALL (semis + diversifiers)
    all_df = ensure_cols(excess, ALL).dropna(how="any")
    if all_df.empty:
        raise ValueError("No overlapping data for ALL in step2_excess_returns.csv")

    # Base = semis (same dates as ALL)
    base_df = all_df[BASE]

    # With benchmark (same dates)
    with_bench = all_df.join(ensure_cols(excess, [BENCHMARK]), how="inner").dropna(how="any")

    # Diagnostics: spans
    def span(df):
        return (df.index.min().date(), df.index.max().date(), df.shape[0], df.shape[1])
    b0, b1, bn, bk = span(base_df)
    a0, a1, an, ak = span(all_df)
    print(f"BASE (semis) window: {b0} → {b1}, n={bn} days, k={bk} assets")
    print(f"ALL  (semis+div)   : {a0} → {a1}, n={an} days, k={ak} assets")
    if BENCHMARK in with_bench.columns:
        w0, w1, wn, wk = span(with_bench)
        print(f"WITH_BENCH (+{BENCHMARK}): {w0} → {w1}, n={wn} days, k={wk} assets")

    # === Correlations / vols ===
    corr_base = save_corr_heatmap(base_df, OUTDIR / "step5d_corr_base.png",
                                  "Correlation Matrix (BASE = Semis)")
    corr_base.to_csv(OUTDIR / "step5d_corr_base.csv")
    med_base = offdiag_median_corr(base_df)
    (OUTDIR / "step5d_median_corr_base.tex").write_text(f"{med_base:.2f}")
    write_vols_table(base_df, OUTDIR / "step5d_vols_corrs_base.tex", med_base)

    corr_all = save_corr_heatmap(all_df, OUTDIR / "step5d_corr_all.png",
                                 "Correlation Matrix (ALL = Semis + Diversifiers)")
    corr_all.to_csv(OUTDIR / "step5d_corr_all.csv")
    med_all = offdiag_median_corr(all_df)
    (OUTDIR / "step5d_median_corr_all.tex").write_text(f"{med_all:.2f}")
    write_vols_table(all_df, OUTDIR / "step5d_vols_corrs_all.tex", med_all)

    # Optional third figure including SMH for reference
    if BENCHMARK in with_bench.columns:
        corr_bench = save_corr_heatmap(with_bench, OUTDIR / "step5d_corr_with_bench.png",
                                       f"Correlation Matrix (Semis + Diversifiers + {BENCHMARK})")
        corr_bench.to_csv(OUTDIR / "step5d_corr_with_bench.csv")
        med_bench = offdiag_median_corr(with_bench)
        (OUTDIR / "step5d_median_corr_with_bench.tex").write_text(f"{med_bench:.2f}")

    # === New diagnostics: block medians & long-form pairs ===
    semis_set, divers_set = set(SEMIS), set(DIVERSIFIERS)

    base_long = pairwise_longform(base_df)
    all_long  = pairwise_longform(all_df)

    base_blocks = block_medians(base_long, semis_set, divers_set)
    all_blocks  = block_medians(all_long,  semis_set, divers_set)

    # Save long-form pair tables
    base_long.to_csv(OUTDIR / "step5d_pairwise_corrs_base.csv", index=False)
    all_long.to_csv(OUTDIR  / "step5d_pairwise_corrs_all.csv", index=False)

    # Print block medians
    def pblock(tag, d):
        n = d.get("n_pairs_total")
        nb = d.get("n_pairs_by_block", {})
        print(f"{tag} pairs total: {n}, by block: {nb}")
        for k in ["semi-semi","semi-div","div-div"]:
            if k in d:
                print(f"  median {k}: {d[k]:.2f}")

    pblock("BASE", base_blocks)
    pblock("ALL ", all_blocks)

    # Ex-VIXY sensitivity (how much VIXY pulls the median)
    if "VIXY" in all_df.columns:
        all_no_vixy = all_df.drop(columns=["VIXY"])
        med_all_no_vixy = offdiag_median_corr(all_no_vixy)
        print(f"Median off-diagonal correlation (ALL \\ VIXY): {med_all_no_vixy:.2f}")
        (OUTDIR / "step5d_median_corr_all_exVIXY.tex").write_text(f"{med_all_no_vixy:.2f}")

    print(f"Median off-diagonal correlation (BASE=Semis): {med_base:.2f}")
    print(f"Median off-diagonal correlation (ALL =Semis+Diversifiers): {med_all:.2f}")

    # === Risk contributions on INVESTABLE set only (ALL) ===
    # Equal Weight on ALL
    ew_weights = {t: 1.0 / len(all_df.columns) for t in all_df.columns}
    rc_ew = risk_contrib_table(all_df, ew_weights)
    rc_ew.to_csv(OUTDIR / "step5d_risk_contrib_EW.csv")
    rc_ew.to_latex(OUTDIR / "step5d_risk_contrib_EW.tex", float_format="%.3f", escape=True)

    # MCAP-like reference weights (normalize to 1 across ALL names)
    w_ref = pd.Series(FINAL_WEIGHTS, dtype=float).reindex(all_df.columns).fillna(0.0)
    if w_ref.sum() == 0:
        raise ValueError("FINAL_WEIGHTS do not overlap with ALL; please check tickers.")
    w_ref = (w_ref / w_ref.sum()).to_dict()
    rc_mcap = risk_contrib_table(all_df, w_ref)
    rc_mcap.to_csv(OUTDIR / "step5d_risk_contrib_MCAP.csv")
    rc_mcap.to_latex(OUTDIR / "step5d_risk_contrib_MCAP.tex", float_format="%.3f", escape=True)

    # Overleaf “one table”
    rc_mcap.to_latex(OUTDIR / "step5d_risk_contrib_all.tex", float_format="%.3f", escape=True)

    print("✓ Step 5d complete: vols, off-diag medians, heatmaps, risk contribs, and diagnostics written.")

if __name__ == "__main__":
    main()


BASE (semis) window: 2022-08-23 → 2025-08-18, n=749 days, k=7 assets
ALL  (semis+div)   : 2022-08-23 → 2025-08-18, n=749 days, k=10 assets
WITH_BENCH (+SMH): 2022-08-23 → 2025-08-18, n=749 days, k=11 assets
BASE pairs total: 21, by block: {'semi-semi': 21, 'semi-div': 0, 'div-div': 0, 'other': 0}
  median semi-semi: 0.67
ALL  pairs total: 45, by block: {'semi-semi': 21, 'semi-div': 21, 'div-div': 3, 'other': 0}
  median semi-semi: 0.67
  median semi-div: 0.05
  median div-div: -0.01
Median off-diagonal correlation (ALL \ VIXY): 0.46
Median off-diagonal correlation (BASE=Semis): 0.67
Median off-diagonal correlation (ALL =Semis+Diversifiers): 0.17
✓ Step 5d complete: vols, off-diag medians, heatmaps, risk contribs, and diagnostics written.


In [11]:
# === Step 6 — Cell 1: Setup, sample μ/Σ, MCAP benchmark, δ, and equilibrium π ===

from pathlib import Path
import warnings
import numpy as np
import pandas as pd

# --------------------
# Config
# --------------------
OUTDIR = Path("outputs"); OUTDIR.mkdir(parents=True, exist_ok=True)
RET_FILE = OUTDIR / "step2_excess_returns.csv"      # daily EXCESS returns (already net of RF)
MCAP_WEIGHTS_CSV = OUTDIR / "step5_mcap_weights.csv"  # optional benchmark weights from Step 5

FREQ = 252
CUTOFF = None                 # e.g., "2025-06-30" to freeze estimation window
SEMIS = ["NVDA","AMD","ASML","AMAT","TSM","LRCX","INTC"]
DIVERSIFIERS = ["GLD","TLT","VIXY"]
ASSET_ORDER = SEMIS + DIVERSIFIERS

# Fallback MCAPs (bn) if step5 CSV is absent; diversifier sleeve set below
FALLBACK_MCAP = {
    "NVDA": 3000.0, "TSM": 1000.0, "ASML": 300.0, "AMD": 280.0,
    "AMAT": 130.0, "LRCX": 120.0, "INTC": 150.0,
}
DIVERSIFIER_SLEEVE = {"GLD": 0.06, "TLT": 0.06, "VIXY": 0.03}  # total ≈ 15%

# --------------------
# Load returns & select universe
# --------------------
if not RET_FILE.exists():
    raise FileNotFoundError(f"Missing {RET_FILE}. Run Step 2 first.")

ret = pd.read_csv(RET_FILE, index_col=0, parse_dates=True)
if CUTOFF is not None:
    ret = ret.loc[:CUTOFF]

# keep the 10 assets that are actually present
assets = [a for a in ASSET_ORDER if a in ret.columns]
if len(assets) < 5:
    raise ValueError(f"Too few of the target assets found. Present: {assets}")

R = ret[assets].copy()

# --------------------
# Annualised sample μ, Σ
# --------------------
mu_sample = R.mean() * FREQ
Sigma_sample = R.cov() * FREQ

# --------------------
# Build MCAP benchmark weights (CSV preferred; fallback otherwise)
# --------------------
def build_benchmark_weights(assets, returns):
    if MCAP_WEIGHTS_CSV.exists():
        try:
            w = pd.read_csv(MCAP_WEIGHTS_CSV, index_col=0).iloc[:,0]
            w = w.reindex(assets).fillna(0.0)
            s = w.sum()
            if s <= 0:
                warnings.warn("MCAP CSV sum <= 0. Falling back to embedded MCAP logic.")
            else:
                return w / s
        except Exception as e:
            warnings.warn(f"Failed to read MCAP weights CSV ({e}). Falling back.")
    # Fallback: pro-rata semis by FALLBACK_MCAP + modest diversifier sleeve
    w = pd.Series(0.0, index=assets, dtype=float)
    semi_names = [a for a in assets if a in FALLBACK_MCAP]
    sleeve = sum(DIVERSIFIER_SLEEVE.get(a,0.0) for a in assets)
    sleeve = min(sleeve, 0.20)  # safety cap

    if semi_names:
        semi_total = 1.0 - sleeve
        semi_sum = sum(FALLBACK_MCAP[s] for s in semi_names)
        for s in semi_names:
            w.loc[s] = semi_total * (FALLBACK_MCAP[s] / semi_sum)
    for a in assets:
        if a in DIVERSIFIER_SLEEVE:
            w.loc[a] = DIVERSIFIER_SLEEVE[a]
    # normalize
    s = w.sum()
    return w if s == 1.0 else (w / s if s > 0 else pd.Series(1/len(assets), index=assets))

w_mcap = build_benchmark_weights(assets, R)

# --------------------
# Compute δ (Market) and equilibrium π
# --------------------
mu_bench = float(mu_sample.loc[assets] @ w_mcap.values)       # annual excess
sigma2_bench = float(w_mcap.values @ Sigma_sample.loc[assets, assets].values @ w_mcap.values)

delta_market = (mu_bench) / sigma2_bench if sigma2_bench > 0 else 10.0  # RF already netted out in excess returns
deltas = pd.Series({"Trustee": 2.0*delta_market, "Market": delta_market, "Kelly": 0.5*delta_market})

# Equilibrium excess returns: π = δ_Market * Σ * w_mcap
pi_sample = Sigma_sample.loc[assets, assets] @ w_mcap.loc[assets] * deltas["Market"]
pi_sample.name = "pi"

# --------------------
# Write Overleaf artifacts & print quick sanity
# --------------------
(OUTDIR / "step6_delta_levels.tex").write_text(deltas.to_frame("delta").to_latex(escape=False, float_format=lambda x: f"{x: .6f}"))
(OUTDIR / "step6_pi_equilibrium_sample.tex").write_text(pi_sample.to_frame("Value").to_latex(escape=False, float_format=lambda x: f"{x: .6f}"))

print("Universe:", assets)
print("\nBenchmark weights (first few):")
print((w_mcap*100).round(2).astype(str).head(10) + "%")

print(f"\nδ (Market): {delta_market:.6f} | μ_bench: {mu_bench:.4f} | σ_bench: {np.sqrt(sigma2_bench):.4f}")
print("\nπ (sample) — head:")
print(pi_sample.round(6).head())


Universe: ['NVDA', 'AMD', 'ASML', 'AMAT', 'TSM', 'LRCX', 'INTC', 'GLD', 'TLT', 'VIXY']

Benchmark weights (first few):
NVDA    61.65%
AMD      3.91%
ASML      4.1%
AMAT     1.82%
TSM     14.18%
LRCX     1.74%
INTC     1.52%
GLD      3.46%
TLT      6.93%
VIXY     0.69%
Name: 0, dtype: object

δ (Market): 3.727700 | μ_bench: 0.6198 | σ_bench: 0.4078

π (sample) — head:
NVDA    0.798106
AMD     0.577738
ASML    0.461463
AMAT    0.472961
TSM     0.462938
Name: pi, dtype: float64


In [12]:
# === Step 6 — Cell 2: Ledoit–Wolf shrinkage Σ, π under LW, PSD checks ===
import numpy as np
import pandas as pd
from numpy.linalg import eigvalsh

# Uses R, assets, w_mcap, FREQ, deltas, Sigma_sample, mu_sample from Cell 1

def ledoit_wolf_identity_shrinkage(returns: pd.DataFrame):
    """
    Ledoit–Wolf shrinkage toward identity-scaled target F = mu*I, mu=trace(S)/p.
    Returns (Sigma_lw_annualized as DataFrame aligned to returns.columns, alpha).
    """
    X = returns.values - returns.values.mean(axis=0, keepdims=True)
    T, p = X.shape
    if T <= 1:
        raise ValueError("Not enough observations for LW shrinkage.")
    # daily sample covariance (unbiased by population convention for LW derivation)
    S = (X.T @ X) / T

    mu = np.trace(S) / p
    F = mu * np.eye(p)

    # phi-hat: average squared deviation of sample cov elements
    # Using LW 2004 identity-target formula
    X2 = X**2
    term1 = (X2.T @ X2) / T
    phi_mat = term1 - (S**2)
    phi = phi_mat.sum()

    # gamma-hat
    gamma = np.sum((S - F)**2)

    # rho-hat for identity target
    rho = 0.0
    for i in range(p):
        rho += (S[i, i] - mu)**2
    rho = rho

    kappa = (phi - rho) / (gamma + 1e-16)
    alpha = max(0.0, min(1.0, kappa / T))

    Sigma_lw_daily = alpha * F + (1.0 - alpha) * S
    Sigma_lw_annual = Sigma_lw_daily * FREQ
    Sigma_lw_df = pd.DataFrame(Sigma_lw_annual, index=returns.columns, columns=returns.columns)
    return Sigma_lw_df, float(alpha)

# Compute LW Σ on daily excess returns R
Sigma_lw, alpha = ledoit_wolf_identity_shrinkage(R)

# Equilibrium π under LW (δ_Market from Cell 1)
pi_lw = Sigma_lw.loc[assets, assets] @ w_mcap.loc[assets] * deltas["Market"]
pi_lw.name = "pi"

# PSD diagnostics
min_eig_sample = float(eigvalsh(Sigma_sample.loc[assets, assets].values).min())
min_eig_lw     = float(eigvalsh(Sigma_lw.loc[assets, assets].values).min())

# Write Overleaf artifact
(OUTDIR / "step6_pi_equilibrium_lw.tex").write_text(pi_lw.to_frame("Value").to_latex(escape=False, float_format=lambda x: f"{x: .6f}"))

print(f"Ledoit–Wolf alpha (shrink to identity): {alpha:.3f}")
print(f"min eigenvalue Σ_sample: {min_eig_sample:.6e}")
print(f"min eigenvalue Σ_lw    : {min_eig_lw:.6e}")
print("\nπ (LW) — head:")
print(pi_lw.round(6).head())


Ledoit–Wolf alpha (shrink to identity): 0.022
min eigenvalue Σ_sample: 1.461386e-02
min eigenvalue Σ_lw    : 1.873246e-02

π (LW) — head:
NVDA    0.789754
AMD     0.564924
ASML    0.451390
AMAT    0.462242
TSM     0.454507
Name: pi, dtype: float64


In [13]:
# === Step 6 — Cell 3: Views (P, Q, Ω) and τ table ===
import numpy as np
import pandas as pd
from pathlib import Path

# Uses: assets, R, Sigma_sample, OUTDIR, deltas from previous cells

# ---- τ values ----
T_obs = len(R)
tau_values = {"tau_1_over_T": 1.0 / max(T_obs, 1), "tau_0_025": 0.025}
tau_df = pd.Series({"tau_1_over_T": f"1/{T_obs}", "tau_0_025": 0.025}, name="tau")
(OUTDIR / "step6_tau_sensitivity.tex").write_text(tau_df.to_frame().to_latex(escape=False))

# ---- View specs (annualised, excess) ----
VIEW_RELATIVE_EQUIP_VS_FABLESS = 0.03   # +3% p.a.
VIEW_ABS_VIXY = -0.15                   # -15% p.a.
VIEW_ABS_TLT  = 0.03                    # +3% p.a.

CONFIDENCES = {
    "rel_equip_gt_fabless": 0.70,   # v1
    "abs_vixy_negative":    0.85,   # v2
    "abs_tlt_positive":     0.60,   # v3
}

# ---- Build P and Q ----
N = len(assets)
idx = {a:i for i,a in enumerate(assets)}
P = np.zeros((3, N))
Q = np.zeros(3)

# v1: (ASML+AMAT+LRCX)/3 - (NVDA+AMD)/2 = +3%
equip = ["ASML","AMAT","LRCX"]
fab   = ["NVDA","AMD"]
for e in equip:
    if e in idx: P[0, idx[e]] =  1.0/len(equip)
for f in fab:
    if f in idx: P[0, idx[f]] = -1.0/len(fab)
Q[0] = VIEW_RELATIVE_EQUIP_VS_FABLESS

# v2: VIXY absolute = -15%
if "VIXY" in idx: P[1, idx["VIXY"]] = 1.0
Q[1] = VIEW_ABS_VIXY

# v3: TLT absolute = +3%
if "TLT" in idx: P[2, idx["TLT"]] = 1.0
Q[2] = VIEW_ABS_TLT

# ---- Map confidences to Ω for tau = 1/T (write this one to Overleaf) ----
tau = tau_values["tau_1_over_T"]
PSigPt = P @ (tau * Sigma_sample.loc[assets, assets].values) @ P.T
base_diag = np.clip(np.diag(PSigPt), 1e-12, None)

c = np.array([
    CONFIDENCES["rel_equip_gt_fabless"],
    CONFIDENCES["abs_vixy_negative"],
    CONFIDENCES["abs_tlt_positive"],
], dtype=float)
nu = (1.0 - c) / np.clip(c, 1e-8, None)  # lower nu => higher confidence

Omega = np.diag(nu * base_diag)

# ---- Write Overleaf artifacts ----
# P matrix with labels
P_df = pd.DataFrame(P, index=["v1","v2","v3"], columns=assets)
(OUTDIR / "step6_views_P.tex").write_text(P_df.to_latex(escape=False, float_format=lambda x: f"{x: .6f}"))

# Q, confidences, nu, and Omega diag
qo_df = pd.DataFrame({
    "Q (annual)": Q,
    "Confidence": c,
    "nu=(1-c)/c": nu,
    "Omega_diag": np.diag(Omega),
}, index=["v1","v2","v3"])
(OUTDIR / "step6_views_QOmega.tex").write_text(qo_df.to_latex(escape=False, float_format=lambda x: f"{x: .6f}"))

print("Views built.")
print("\nP (rows v1..v3, cols assets):")
print(P_df)

print("\nQ (annual):", np.round(Q, 4))
print("Confidences:", np.round(c, 2), "  nu:", np.round(nu, 3))
print("Omega diag (tau=1/T):", np.round(np.diag(Omega), 6))


Views built.

P (rows v1..v3, cols assets):
    NVDA  AMD      ASML      AMAT  TSM      LRCX  INTC  GLD  TLT  VIXY
v1  -0.5 -0.5  0.333333  0.333333  0.0  0.333333   0.0  0.0  0.0   0.0
v2   0.0  0.0  0.000000  0.000000  0.0  0.000000   0.0  0.0  0.0   1.0
v3   0.0  0.0  0.000000  0.000000  0.0  0.000000   0.0  0.0  1.0   0.0

Q (annual): [ 0.03 -0.15  0.03]
Confidences: [0.7  0.85 0.6 ]   nu: [0.429 0.176 0.667]
Omega diag (tau=1/T): [5.40e-05 1.17e-04 2.40e-05]


In [14]:
# === Step 6 — Cell 4: BL posterior μ (sample Σ, τ in {1/T, 0.025}) ===
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Uses from previous cells: assets, R, mu_sample, Sigma_sample, pi_sample,
# P, Q, CONFIDENCES, tau_values, OUTDIR

def build_omega(P, Sigma, tau, confidences):
    """Diagonal Ω using nu=(1-c)/c scaled by diag(P (tau Σ) P^T)."""
    PSigPt = P @ (tau * Sigma) @ P.T
    base_diag = np.clip(np.diag(PSigPt), 1e-12, None)
    c = np.array([confidences["rel_equip_gt_fabless"],
                  confidences["abs_vixy_negative"],
                  confidences["abs_tlt_positive"]], dtype=float)
    nu = (1.0 - c) / np.clip(c, 1e-8, None)
    return np.diag(nu * base_diag)

def bl_posterior(mu_prior, Sigma, P, Q, Omega, tau):
    """Standard BL closed form."""
    Sigma_tau_inv = np.linalg.inv(tau * Sigma)
    mid = Sigma_tau_inv + P.T @ np.linalg.inv(Omega) @ P
    rhs = Sigma_tau_inv @ mu_prior + P.T @ np.linalg.inv(Omega) @ Q
    return np.linalg.solve(mid, rhs)

# Compute posterior for each tau
posteriors = {}
for tau_name, tau_val in tau_values.items():
    Omega_tau = build_omega(P, Sigma_sample.loc[assets, assets].values, tau_val, CONFIDENCES)
    mu_post = bl_posterior(
        mu_prior=pi_sample.loc[assets].values,
        Sigma=Sigma_sample.loc[assets, assets].values,
        P=P,
        Q=Q,
        Omega=Omega_tau,
        tau=tau_val
    )
    s = pd.Series(mu_post, index=assets, name=f"Posterior ({tau_name})")
    posteriors[tau_name] = s
    # Write individual vectors for Overleaf
    if tau_name == "tau_1_over_T":
        (OUTDIR / "step6_posterior_mu_sample_tau1overT.tex").write_text(
            s.to_frame("Value").to_latex(escape=False, float_format=lambda x: f"{x: .6f}")
        )
    else:
        (OUTDIR / "step6_posterior_mu_sample_tau_0_025.tex").write_text(
            s.to_frame("Value").to_latex(escape=False, float_format=lambda x: f"{x: .6f}")
        )

# Comparison table (Naive mean vs π vs Posterior columns)
cmp = pd.DataFrame({
    "Naive (mean)": mu_sample.loc[assets],
    "Equilibrium π": pi_sample.loc[assets],
    "Posterior (τ=1/T)": posteriors["tau_1_over_T"],
    "Posterior (τ=0.025)": posteriors["tau_0_025"],
})
(OUTDIR / "step6_mu_compare.tex").write_text(cmp.to_latex(escape=False, float_format=lambda x: f"{x: .6f}"))

# Quick sanity: bar chart of Naive vs π vs Posterior (τ=1/T)
plt.figure(figsize=(9,3.8))
width = 0.25
x = np.arange(len(assets))
plt.bar(x - width, cmp["Naive (mean)"].values, width, label="Naive")
plt.bar(x,         cmp["Equilibrium π"].values, width, label="π")
plt.bar(x + width, cmp["Posterior (τ=1/T)"].values, width, label="Posterior")
plt.xticks(x, assets, rotation=45, ha="right")
plt.ylabel("Annualised excess return")
plt.title("BL: Naive vs Equilibrium π vs Posterior (sample Σ)")
plt.legend()
plt.tight_layout()
plt.savefig(OUTDIR / "step6_mu_bar_sample.png"); plt.close()

# Console sanity
print("Posterior μ (τ=1/T) — head:\n", posteriors["tau_1_over_T"].round(6).head())
print("\nCorrelations among vectors:")
print(pd.DataFrame({
    "Naive_vs_pi": cmp["Naive (mean)"].corr(cmp["Equilibrium π"]),
    "Naive_vs_Post(1/T)": cmp["Naive (mean)"].corr(cmp["Posterior (τ=1/T)"]),
    "pi_vs_Post(1/T)": cmp["Equilibrium π"].corr(cmp["Posterior (τ=1/T)"]),
}, index=["corr"]).T.round(3))
print("\nWrote: step6_posterior_mu_sample_tau1overT.tex, step6_posterior_mu_sample_tau_0_025.tex, step6_mu_compare.tex, step6_mu_bar_sample.png")


Posterior μ (τ=1/T) — head:
 NVDA    0.538897
AMD     0.352109
ASML    0.395745
AMAT    0.405588
TSM     0.343651
Name: Posterior (tau_1_over_T), dtype: float64

Correlations among vectors:
                     corr
Naive_vs_pi         0.894
Naive_vs_Post(1/T)  0.860
pi_vs_Post(1/T)     0.975

Wrote: step6_posterior_mu_sample_tau1overT.tex, step6_posterior_mu_sample_tau_0_025.tex, step6_mu_compare.tex, step6_mu_bar_sample.png


In [15]:
# === Step 6 — Cell 5: optimiser scaffolding + Min-Variance (sample Σ) ===
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.optimize import minimize

# Uses: assets, Sigma_sample, OUTDIR, w_mcap from previous cells

# ---- Bounds (long-only + caps) ----
CAP_SEMI = 0.25
CAP_GLD_TLT = 0.30
CAP_VIXY = 0.05

def make_bounds(assets):
    b = []
    for a in assets:
        if a == "VIXY":
            b.append((0.0, CAP_VIXY))
        elif a in ("GLD","TLT"):
            b.append((0.0, CAP_GLD_TLT))
        else:  # semis
            b.append((0.0, CAP_SEMI))
    return tuple(b)

bounds = make_bounds(assets)

# ---- Min-Variance solver (budget = 1) ----
def min_variance(Sigma, bounds):
    p = Sigma.shape[0]
    def obj(w): return float(w @ Sigma @ w)
    cons = [{"type":"eq", "fun": lambda w: np.sum(w) - 1.0}]
    x0 = np.full(p, 1.0/p)
    res = minimize(obj, x0=x0, method="SLSQP", bounds=bounds, constraints=cons,
                   options={"maxiter": 2000, "ftol": 1e-12})
    return res

Sigma_mat = Sigma_sample.loc[assets, assets].values
res = min_variance(Sigma_mat, bounds)

if not res.success:
    print("Min-Variance failed:", res.message)
    # simple fallback: clip equal weights to bounds and renormalize
    w = np.full(len(assets), 1.0/len(assets))
    w = np.minimum(w, np.array([ub for (_,ub) in bounds]))
    w = w / w.sum()
else:
    w = res.x

w_minvar_sample = pd.Series(w, index=assets, name="MinVar (sample Σ)")

# ---- Write Overleaf table ----
(OUTDIR / "step6_w_MinVar_sample.tex").write_text(
    (w_minvar_sample*100).to_frame("% Weight").to_latex(escape=False, float_format=lambda x: f"{x: .2f}")
)

# ---- Quick sanity prints ----
sumw = float(w_minvar_sample.sum())
viol_lo = (w_minvar_sample < -1e-9).sum()
viol_hi = sum((w_minvar_sample.values - np.array([ub for (_,ub) in bounds])) > 1e-9)
port_vol = np.sqrt(float(w_minvar_sample.values @ Sigma_mat @ w_minvar_sample.values))

print("Min-Var — summary")
print(f"  Sum of weights: {sumw:.6f}")
print(f"  Lower bound violations: {viol_lo} | Upper bound violations: {viol_hi}")
print(f"  Annualised volatility: {port_vol:.4f}")
print("\nWeights (%):")
print((w_minvar_sample*100).round(2))


Min-Var — summary
  Sum of weights: 1.000000
  Lower bound violations: 0 | Upper bound violations: 0
  Annualised volatility: 0.1383

Weights (%):
NVDA     0.00
AMD      0.00
ASML     2.01
AMAT     5.17
TSM     18.79
LRCX     0.00
INTC     9.03
GLD     30.00
TLT     30.00
VIXY     5.00
Name: MinVar (sample Σ), dtype: float64


In [16]:
# === Step 6 — Cell 6: Mean–Variance (Trustee/Market/Kelly) with BL posterior μ (τ=1/T, sample Σ) ===
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from pathlib import Path

# Uses from previous cells: assets, Sigma_sample, deltas, OUTDIR,
# posteriors (dict from Cell 4), w_mcap, bounds (from Cell 5)

# ----- Config: optional exposure floor on the semi sleeve -----
SEMI_FLOOR = 0.0   # set to 0.60 if you want "sum of semis ≥ 60%"

semi_idx = [i for i,a in enumerate(assets) if a not in ("GLD","TLT","VIXY")]

def mean_variance(mu, Sigma, delta, bounds, semi_floor=0.0):
    mu = np.asarray(mu); Sigma = np.asarray(Sigma)
    p = len(mu)
    def obj(w):  # 0.5*δ*w'Σw − μ'w
        return 0.5 * delta * float(w @ Sigma @ w) - float(mu @ w)
    cons = [{"type":"eq", "fun": lambda w: np.sum(w) - 1.0}]
    if semi_floor > 0 and len(semi_idx) > 0:
        cons.append({"type":"ineq", "fun": lambda w: np.sum(w[semi_idx]) - semi_floor})
    x0 = np.full(p, 1.0/p)
    res = minimize(obj, x0=x0, method="SLSQP", bounds=bounds, constraints=cons,
                   options={"maxiter": 4000, "ftol": 1e-12})
    return res

# Posterior μ (τ=1/T)
mu_post = posteriors["tau_1_over_T"].loc[assets].values
Sigma_mat = Sigma_sample.loc[assets, assets].values

def write_weights_tex(name, w_vec):
    s = pd.Series(w_vec, index=assets, name=name)
    (OUTDIR / f"step6_w_MV_sample_{name}.tex").write_text(
        (s*100).to_frame("% Weight").to_latex(escape=False, float_format=lambda x: f"{x: .2f}")
    )
    return s

summ_rows = []
for name, delta in deltas.items():
    res = mean_variance(mu_post, Sigma_mat, delta, bounds, semi_floor=SEMI_FLOOR)
    if not res.success:
        print(f"MVO ({name}) failed: {res.message}")
        continue
    w = res.x
    s = write_weights_tex(name, w)

    port_mu = float(mu_post @ w)
    port_vol = float(np.sqrt(w @ Sigma_mat @ w))
    sr = port_mu / (port_vol + 1e-16)
    semi_sum = float(np.sum(w[semi_idx]))
    bounds_hi = np.array([ub for (_,ub) in bounds])
    ub_hits = int(np.sum((w > bounds_hi - 1e-6)))
    lb_hits = int(np.sum((w < 1e-6)))
    summ_rows.append([name, port_mu, port_vol, sr, semi_sum, ub_hits, lb_hits])

summary = pd.DataFrame(summ_rows, columns=["Delta", "AnnExpRet", "AnnVol", "Sharpe", "SemiSleeve", "UpperBoundHits", "NearZeroWeights"])
print("\nMVO summary (sample Σ, τ=1/T):")
print(summary.round(4))

# Also print top-5 weights vs benchmark for the Market posture to sanity-check NVDA/AMD exposure
if "Market" in deltas.index:
    res_mkt = mean_variance(mu_post, Sigma_mat, deltas["Market"], bounds, semi_floor=SEMI_FLOOR)
    if res_mkt.success:
        w_mkt = pd.Series(res_mkt.x, index=assets)
        df = pd.DataFrame({"MVO_Market": w_mkt, "MCAP_Bench": w_mcap.loc[assets]})
        df["Active"] = df["MVO_Market"] - df["MCAP_Bench"]
        print("\nMVO (Market) — top 5 by weight:")
        print((df.sort_values("MVO_Market", ascending=False).head(5) * 100).round(2))
        print("\nMVO (Market) — semi sleeve %:", round(w_mkt.iloc[semi_idx].sum()*100, 2))



MVO summary (sample Σ, τ=1/T):
     Delta  AnnExpRet  AnnVol  Sharpe  SemiSleeve  UpperBoundHits  \
0  Trustee     0.1983  0.1797  1.1034      0.4393               1   
1   Market     0.3316  0.2922  1.1349      0.7645               2   
2    Kelly     0.4365  0.4016  1.0868      1.0000               3   

   NearZeroWeights  
0                2  
1                2  
2                5  

MVO (Market) — top 5 by weight:
      MVO_Market  MCAP_Bench  Active
NVDA       25.00       61.65  -36.65
ASML       15.87        4.10   11.77
AMAT       14.13        1.82   12.31
TLT        11.49        6.93    4.56
TSM        11.16       14.18   -3.03

MVO (Market) — semi sleeve %: 76.45


In [17]:
# === Step 6 — Cell 7: Max Sharpe (sample Σ, τ=1/T) ===
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from pathlib import Path

# Uses: assets, OUTDIR, bounds (from Cell 5)
#       posteriors (from Cell 4), Sigma_sample, w_mcap

mu_post = posteriors["tau_1_over_T"].loc[assets].values
Sigma_mat = Sigma_sample.loc[assets, assets].values

def sharpe(mu, Sigma, w):
    num = float(mu @ w)
    den = np.sqrt(float(w @ Sigma @ w))
    return num / (den + 1e-16)

def max_sharpe(mu, Sigma, bounds):
    p = len(mu)
    def obj(w):  # maximize Sharpe -> minimize negative Sharpe
        return -sharpe(mu, Sigma, w)
    cons = [{"type":"eq", "fun": lambda w: np.sum(w) - 1.0}]
    x0 = np.full(p, 1.0/p)
    res = minimize(obj, x0=x0, method="SLSQP", bounds=bounds, constraints=cons,
                   options={"maxiter": 4000, "ftol": 1e-12})
    return res

res = max_sharpe(mu_post, Sigma_mat, bounds)
if not res.success:
    print("Max Sharpe failed:", res.message)
else:
    w_ms = pd.Series(res.x, index=assets, name="MaxSharpe (sample Σ)")
    # Write Overleaf
    (OUTDIR / "step6_w_MS_sample_Market.tex").write_text(
        (w_ms*100).to_frame("% Weight").to_latex(escape=False, float_format=lambda x: f"{x: .2f}")
    )

    # Active bar vs benchmark
    active = (w_ms - w_mcap.loc[assets])
    plt.figure(figsize=(9,3.4))
    plt.bar(active.index, active.values)
    plt.xticks(rotation=45, ha="right")
    plt.ylabel("Active Weight")
    plt.title("Active Weights: Max Sharpe (sample Σ, τ=1/T)")
    plt.tight_layout()
    plt.savefig(OUTDIR / "step6_active_MS_Market.png"); plt.close()

    # Summary stats
    port_mu = float(mu_post @ w_ms.values)
    port_vol = float(np.sqrt(w_ms.values @ Sigma_mat @ w_ms.values))
    sr = port_mu / (port_vol + 1e-16)
    semi_idx = [i for i,a in enumerate(assets) if a not in ("GLD","TLT","VIXY")]
    semi_sum = float(w_ms.values[semi_idx].sum())

    bounds_hi = np.array([ub for (_,ub) in bounds])
    ub_hits = int(np.sum((w_ms.values > bounds_hi - 1e-6)))
    lb_hits = int(np.sum((w_ms.values < 1e-6)))

    print("Max Sharpe — summary")
    print(f"  AnnExpRet: {port_mu:.4f} | AnnVol: {port_vol:.4f} | Sharpe: {sr:.3f} | Semi sleeve: {semi_sum*100:.2f}%")
    print(f"  UpperBoundHits: {ub_hits} | NearZeroWeights: {lb_hits}")
    print("\nWeights (%):")
    print((w_ms*100).round(2))

    # Top-5 vs benchmark
    comp = pd.DataFrame({"MS": w_ms, "MCAP": w_mcap.loc[assets]})
    comp["Active"] = comp["MS"] - comp["MCAP"]
    print("\nTop 5 by MS weight (%):")
    print((comp.sort_values("MS", ascending=False).head(5)*100).round(2))


Max Sharpe — summary
  AnnExpRet: 0.2962 | AnnVol: 0.2603 | Sharpe: 1.138 | Semi sleeve: 66.60%
  UpperBoundHits: 2 | NearZeroWeights: 2

Weights (%):
NVDA    25.00
AMD      0.00
ASML    12.87
AMAT    11.70
TSM      8.62
LRCX     8.40
INTC     0.00
GLD     12.98
TLT     15.43
VIXY     5.00
Name: MaxSharpe (sample Σ), dtype: float64

Top 5 by MS weight (%):
         MS   MCAP  Active
NVDA  25.00  61.65  -36.65
TLT   15.43   6.93    8.50
GLD   12.98   3.46    9.51
ASML  12.87   4.10    8.77
AMAT  11.70   1.82    9.88


In [19]:
# === Step 6 — Cell 8 (fixed): Frontiers (π vs BL μ) + Active Risk (MVO Market) ===
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from pathlib import Path

# Uses from earlier cells: assets, OUTDIR, Sigma_sample, pi_sample, posteriors, w_mcap, bounds,
# and (ideally) mu_sample. No reading .tex files here.

Sigma_mat = Sigma_sample.loc[assets, assets].values
mu_pi     = pi_sample.loc[assets].values                      # prior (equilibrium)
mu_post   = posteriors["tau_1_over_T"].loc[assets].values     # posterior (with views)

def min_var_at_target(mu_target, mu_vec, Sigma, bounds):
    p = len(mu_vec)
    def obj(w): return float(w @ Sigma @ w)
    cons = [
        {"type":"eq", "fun": lambda w: np.sum(w) - 1.0},
        {"type":"eq", "fun": lambda w: float(mu_vec @ w) - float(mu_target)},
    ]
    x0 = np.full(p, 1.0/p)
    return minimize(obj, x0=x0, method="SLSQP", bounds=bounds, constraints=cons,
                    options={"maxiter": 4000, "ftol": 1e-12})

def make_frontier(mu_vec, Sigma, bounds, n=45):
    mu_lo, mu_hi = float(mu_vec.min()), float(mu_vec.max())
    targets = np.linspace(mu_lo*0.5, mu_hi*1.05, n)
    xs, ys = [], []
    for t in targets:
        res = min_var_at_target(t, mu_vec, Sigma, bounds)
        if res.success:
            w = res.x
            xs.append(np.sqrt(float(w @ Sigma @ w)))
            ys.append(float(mu_vec @ w))
    return np.array(xs), np.array(ys)

# Build both frontiers
x_pi,   y_pi   = make_frontier(mu_pi,   Sigma_mat, bounds)
x_post, y_post = make_frontier(mu_post, Sigma_mat, bounds)

plt.figure(figsize=(6.0,4.4))
plt.plot(x_pi,   y_pi,   label="Frontier (π prior)")
plt.plot(x_post, y_post, label="Frontier (BL posterior)")
plt.xlabel("Volatility"); plt.ylabel("Expected excess return")
plt.title("Efficient Frontiers (sample Σ)")
plt.legend()
plt.tight_layout()
plt.savefig(OUTDIR / "step6_frontier_sample.png"); plt.close()

# Recompute δ_Market safely (prefer in-memory 'deltas', else recompute from μ, Σ, w_mcap)
def get_delta_market():
    try:
        return float(deltas["Market"])  # from Cell 1
    except Exception:
        mu_bench = float(mu_sample.loc[assets] @ w_mcap.loc[assets].values)  # annual excess
        sigma2_bench = float(w_mcap.loc[assets].values @ Sigma_mat @ w_mcap.loc[assets].values)
        return (mu_bench / sigma2_bench) if sigma2_bench > 0 else 10.0

delta_market = get_delta_market()

# MVO (Market) with posterior μ
def mvo(mu_vec, Sigma, delta, bounds):
    p = len(mu_vec)
    def obj(w): return 0.5*delta*float(w @ Sigma @ w) - float(mu_vec @ w)
    cons = [{"type":"eq", "fun": lambda w: np.sum(w) - 1.0}]
    x0 = np.full(p, 1.0/p)
    return minimize(obj, x0=x0, method="SLSQP", bounds=bounds, constraints=cons,
                    options={"maxiter": 4000, "ftol": 1e-12})

res = mvo(mu_post, Sigma_mat, delta_market, bounds)
if not res.success:
    raise RuntimeError(f"MVO (Market) failed: {res.message}")

w_mvo = pd.Series(res.x, index=assets, name="MVO_Market")

# Active stats vs benchmark
d = w_mvo.values - w_mcap.loc[assets].values
te = np.sqrt(float(d @ Sigma_mat @ d))
act_mu = float(mu_post @ d)
act_sr = act_mu / (te + 1e-16)

active_tbl = pd.DataFrame(
    {"Active TE": [te], "Active ExpRet": [act_mu], "Active Sharpe": [act_sr]},
    index=["sample"]
)
(OUTDIR / "step6_active_risk_MV_Market.tex").write_text(
    active_tbl.to_latex(escape=False, float_format=lambda x: f"{x: .4f}")
)

# Active bar plot
plt.figure(figsize=(9,3.4))
active = (w_mvo - w_mcap.loc[assets])
plt.bar(active.index, active.values)
plt.xticks(rotation=45, ha="right")
plt.ylabel("Active Weight")
plt.title("Active Weights: MVO (Market, sample Σ)")
plt.tight_layout()
plt.savefig(OUTDIR / "step6_active_MV_Market.png"); plt.close()

print("Wrote: step6_frontier_sample.png, step6_active_risk_MV_Market.tex, step6_active_MV_Market.png")
print(f"Active TE={te:.4f}, Active ExpRet={act_mu:.4f}, Active Sharpe={act_sr:.3f}")


Wrote: step6_frontier_sample.png, step6_active_risk_MV_Market.tex, step6_active_MV_Market.png
Active TE=0.1639, Active ExpRet=-0.0988, Active Sharpe=-0.603


In [20]:
# === Step 6 — Cell 9: No-rebalancing backtest + rolling betas ===
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Reuse globals where possible; also reload what we need so this cell is standalone.
OUTDIR = Path("outputs"); OUTDIR.mkdir(parents=True, exist_ok=True)
RET_FILE = OUTDIR / "step2_excess_returns.csv"
FACTORS_FILE = OUTDIR / "step4_all_factors.csv"

# ---- Load returns (daily excess) and align universe ----
ret_all = pd.read_csv(RET_FILE, index_col=0, parse_dates=True)
assets = [c for c in ret_all.columns if c in ["NVDA","AMD","ASML","AMAT","TSM","LRCX","INTC","GLD","TLT","VIXY"]]
R = ret_all[assets].copy()

FREQ = 252

# ---- Pull μ_post (τ=1/T) and Σ_sample from earlier cells; recompute if missing ----
try:
    mu_post_vec = posteriors["tau_1_over_T"].loc[assets].values
    Sigma_mat = Sigma_sample.loc[assets, assets].values
except NameError:
    # Quick recompute (rare, only if notebook state was reset)
    mu_sample = R.mean()*FREQ
    Sigma_sample = R.cov()*FREQ
    # Fall back: use equilibrium π as prior and a neutral posterior (no views)
    mu_post_vec = mu_sample.loc[assets].values
    Sigma_mat = Sigma_sample.loc[assets, assets].values

# ---- Bounds (must match earlier) ----
CAP_SEMI, CAP_GLD_TLT, CAP_VIXY = 0.25, 0.30, 0.05
def make_bounds(assets):
    b = []
    for a in assets:
        if a == "VIXY":
            b.append((0.0, CAP_VIXY))
        elif a in ("GLD","TLT"):
            b.append((0.0, CAP_GLD_TLT))
        else:
            b.append((0.0, CAP_SEMI))
    return tuple(b)
bounds = make_bounds(assets)

# ---- Benchmark weights (reuse if in memory; else rebuild fallback) ----
try:
    w_bench = w_mcap.loc[assets]
except NameError:
    # fallback logic from Cell 1
    FALLBACK_MCAP = {"NVDA":3000.0,"TSM":1000.0,"ASML":300.0,"AMD":280.0,"AMAT":130.0,"LRCX":120.0,"INTC":150.0}
    DIVERSIFIER_SLEEVE = {"GLD":0.06,"TLT":0.06,"VIXY":0.03}
    w = pd.Series(0.0, index=assets, dtype=float)
    semi_names = [a for a in assets if a in FALLBACK_MCAP]
    sleeve = sum(DIVERSIFIER_SLEEVE.get(a,0.0) for a in assets); sleeve = min(sleeve, 0.20)
    if semi_names:
        semi_total = 1.0 - sleeve
        semi_sum = sum(FALLBACK_MCAP[s] for s in semi_names)
        for s in semi_names:
            w.loc[s] = semi_total * (FALLBACK_MCAP[s]/semi_sum)
    for a in assets:
        if a in DIVERSIFIER_SLEEVE: w.loc[a] = DIVERSIFIER_SLEEVE[a]
    w_bench = w / w.sum()

# ---- Solvers (MVO with δ_Market; Max Sharpe) ----
def get_delta_market(mu_post_vec, Sigma_mat, w_bench):
    mu_b = float(mu_post_vec @ w_bench.values)  # use posterior μ for consistency
    sig2_b = float(w_bench.values @ Sigma_mat @ w_bench.values)
    return (mu_b / sig2_b) if sig2_b > 0 else 10.0

def mean_variance(mu, Sigma, delta, bounds):
    p = len(mu)
    def obj(w): return 0.5*delta*float(w @ Sigma @ w) - float(mu @ w)
    cons = [{"type":"eq", "fun": lambda w: np.sum(w) - 1.0}]
    x0 = np.full(p, 1.0/p)
    return minimize(obj, x0=x0, method="SLSQP", bounds=bounds, constraints=cons,
                    options={"maxiter": 4000, "ftol": 1e-12})

def max_sharpe(mu, Sigma, bounds):
    p = len(mu)
    def shp(w):
        num = float(mu @ w); den = np.sqrt(float(w @ Sigma @ w))
        return num/(den+1e-16)
    def obj(w): return -shp(w)
    cons = [{"type":"eq", "fun": lambda w: np.sum(w) - 1.0}]
    x0 = np.full(p, 1.0/p)
    return minimize(obj, x0=x0, method="SLSQP", bounds=bounds, constraints=cons,
                    options={"maxiter": 4000, "ftol": 1e-12})

delta_mkt = get_delta_market(mu_post_vec, Sigma_mat, w_bench)

res_mvo = mean_variance(mu_post_vec, Sigma_mat, delta_mkt, bounds)
res_ms  = max_sharpe(mu_post_vec, Sigma_mat, bounds)
if (not res_mvo.success) or (not res_ms.success):
    raise RuntimeError(f"Optimiser failed. MVO success={res_mvo.success}, MS success={res_ms.success}")

w_mvo = pd.Series(res_mvo.x, index=assets, name="BL_MVO")
w_ms  = pd.Series(res_ms.x,  index=assets, name="BL_MS")

# ---- Build portfolio daily excess returns (fixed weights; no rebalancing) ----
port_mvo = (R @ w_mvo).rename("BL_MVO")
port_ms  = (R @ w_ms).rename("BL_MS")
w_ew = pd.Series(1.0/len(assets), index=assets)
ew_ret = (R @ w_ew).rename("EW")
mcap_ret = (R @ w_bench).rename("MCAP")
smh_ret = ret_all["SMH"] if "SMH" in ret_all.columns else None

# ---- Performance stats & table ----
def perf_stats(ret: pd.Series):
    r = ret.dropna()
    if r.empty:
        return dict(AnnRet=np.nan, AnnVol=np.nan, Sharpe=np.nan, MaxDD=np.nan)
    mu = r.mean()*FREQ
    vol = r.std()*np.sqrt(FREQ)
    sr = mu/vol if vol>0 else np.nan
    curve = (1+r).cumprod()
    dd = (curve/curve.cummax() - 1.0).min()
    return dict(AnnRet=float(mu), AnnVol=float(vol), Sharpe=float(sr), MaxDD=float(dd))

perf = {
    "BL_MVO": perf_stats(port_mvo),
    "BL_MS":  perf_stats(port_ms),
    "EW":     perf_stats(ew_ret),
    "MCAP":   perf_stats(mcap_ret),
}
if smh_ret is not None:
    perf["SMH"] = perf_stats(smh_ret)

df_perf = pd.DataFrame(perf).T
df_perf_fmt = df_perf.copy()
df_perf_fmt["AnnRet"] *= 100; df_perf_fmt["AnnVol"] *= 100; df_perf_fmt["MaxDD"] *= 100
df_perf_fmt.columns = ["AnnRet (%)","AnnVol (%)","Sharpe","MaxDD (%)"]
(OUTDIR / "step6_perf_bl_vs_bench.tex").write_text(
    df_perf_fmt.to_latex(escape=False, float_format=lambda x: f"{x: .2f}")
)

# ---- Cumulative $10k chart ----
def growth_of_10k(series): return 10000*(1+series.fillna(0)).cumprod()

plt.figure(figsize=(8.4,4.2))
plt.plot(growth_of_10k(port_mvo), label="BL_MVO")
plt.plot(growth_of_10k(port_ms),  label="BL_MS")
plt.plot(growth_of_10k(ew_ret),   label="EW")
plt.plot(growth_of_10k(mcap_ret), label="MCAP")
if smh_ret is not None: plt.plot(growth_of_10k(smh_ret), label="SMH")
plt.legend(); plt.title("Cumulative P&L (No Rebalancing)")
plt.tight_layout()
plt.savefig(OUTDIR / "step6_cum_bl_vs_bench.png"); plt.close()

# ---- Rolling betas (optional) ----
def rolling_beta(y: pd.Series, x: pd.Series, window=252):
    df = pd.concat([y, x], axis=1).dropna()
    if len(df) < window+5: return pd.Series(dtype=float)
    betas, idxs = [], []
    for i in range(window, len(df)):
        sl = df.iloc[i-window:i]
        X, Y = sl.iloc[:,1].values, sl.iloc[:,0].values
        vx = np.var(X)
        b = np.cov(X, Y, ddof=0)[0,1] / (vx + 1e-16)
        betas.append(b); idxs.append(df.index[i])
    return pd.Series(betas, index=idxs)

# vs SMH, if available
if smh_ret is not None:
    beta_smh = rolling_beta(port_mvo, smh_ret)
    if not beta_smh.empty:
        plt.figure(figsize=(7.6,3.4))
        plt.plot(beta_smh)
        plt.title("Rolling 252-day Beta: BL MVO vs SMH")
        plt.tight_layout()
        plt.savefig(OUTDIR / "step6_rolling_beta_BL_SMH.png"); plt.close()

# vs SemiCapex (if factor file present)
try:
    factors = pd.read_csv(FACTORS_FILE, index_col=0, parse_dates=True)
    if "SemiCapex" in factors.columns:
        semi_beta = rolling_beta(port_mvo, factors["SemiCapex"])
        if not semi_beta.empty:
            plt.figure(figsize=(7.6,3.4))
            plt.plot(semi_beta)
            plt.title("Rolling 252-day Beta: BL MVO vs SemiCapex")
            plt.tight_layout()
            plt.savefig(OUTDIR / "step6_rolling_beta_BL_SemiCapex.png"); plt.close()
except Exception:
    pass

print("Wrote: step6_perf_bl_vs_bench.tex, step6_cum_bl_vs_bench.png",
      "(+ rolling beta PNGs if sources available)")


Wrote: step6_perf_bl_vs_bench.tex, step6_cum_bl_vs_bench.png (+ rolling beta PNGs if sources available)


In [22]:
# === Step 6 — Cell 10: LW posterior μ, LW frontier, MVO(Market) under LW, view stability ===
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from pathlib import Path

OUTDIR = Path("outputs"); OUTDIR.mkdir(parents=True, exist_ok=True)

# Reuse variables if present; otherwise reconstruct minimal pieces
try:
    assets
    Sigma_sample
    Sigma_lw
    posteriors
    P, Q
    tau_values
    w_mcap
    bounds
except NameError:
    # Minimal reloads (assumes prior cells were run in this session)
    raise RuntimeError("Some earlier variables are missing; please run Cells 1–3 and 5–7 first.")

# --- build Omega for LW using tau=1/T ---
T_obs = len(pd.read_csv(OUTDIR / "step2_excess_returns.csv", index_col=0, parse_dates=True))
tau = 1.0 / max(T_obs, 1)

def build_omega(P, Sigma, tau, confidences):
    c = np.array([confidences["rel_equip_gt_fabless"],
                  confidences["abs_vixy_negative"],
                  confidences["abs_tlt_positive"]], dtype=float)
    nu = (1.0 - c) / np.clip(c, 1e-8, None)
    PSigPt = P @ (tau * Sigma) @ P.T
    base_diag = np.clip(np.diag(PSigPt), 1e-12, None)
    return np.diag(nu * base_diag)

CONFIDENCES = {
    "rel_equip_gt_fabless": 0.70,
    "abs_vixy_negative":    0.85,
    "abs_tlt_positive":     0.60,
}

def bl_posterior(mu_prior, Sigma, P, Q, Omega, tau):
    Sigma_tau_inv = np.linalg.inv(tau * Sigma)
    mid = Sigma_tau_inv + P.T @ np.linalg.inv(Omega) @ P
    rhs = Sigma_tau_inv @ mu_prior + P.T @ np.linalg.inv(Omega) @ Q
    return np.linalg.solve(mid, rhs)

# Priors (π) already computed for both Σ; reuse
pi_lw_vec = (Sigma_lw.loc[assets, assets] @ w_mcap.loc[assets] * float(pd.read_csv(OUTDIR / "step6_delta_levels.tex", sep="&", header=None, engine="python", on_bad_lines="skip").iloc[1,2])
             if False else (Sigma_lw.loc[assets, assets] @ w_mcap.loc[assets] * posteriors["tau_1_over_T"].name if False else None))
# Safer: recompute delta_market from memory (created in Cell 1/6/8)
try:
    delta_market = float(deltas["Market"])
except Exception:
    # rebuild from sample μ/Σ and w_mcap
    mu_sample = pd.read_csv(OUTDIR / "step2_excess_returns.csv", index_col=0, parse_dates=True)[assets].mean()*252
    Sigma_sample_tmp = pd.read_csv(OUTDIR / "step2_excess_returns.csv", index_col=0, parse_dates=True)[assets].cov()*252
    mu_b = float(mu_sample.values @ w_mcap.loc[assets].values)
    sig2_b = float(w_mcap.loc[assets].values @ Sigma_sample_tmp.values @ w_mcap.loc[assets].values)
    delta_market = (mu_b/sig2_b) if sig2_b>0 else 10.0

pi_lw = Sigma_lw.loc[assets, assets] @ w_mcap.loc[assets] * delta_market
pi_lw = pd.Series(pi_lw, index=assets, name="pi_lw")

# Posterior μ under LW, tau=1/T
Omega_lw = build_omega(P, Sigma_lw.loc[assets, assets].values, tau, CONFIDENCES)
mu_post_lw = bl_posterior(
    mu_prior=pi_lw.values,
    Sigma=Sigma_lw.loc[assets, assets].values,
    P=P, Q=Q, Omega=Omega_lw, tau=tau
)
mu_post_lw = pd.Series(mu_post_lw, index=assets, name="Posterior_LW_tau_1_over_T")
(OUTDIR / "step6_posterior_mu_lw_tau1overT.tex").write_text(
    mu_post_lw.to_frame("Value").to_latex(escape=False, float_format=lambda x: f"{x: .6f}")
)

# View-stability table: compare sample vs LW posterior
mu_post_sample = posteriors["tau_1_over_T"].loc[assets]
stab = pd.DataFrame({
    "Pearson": [mu_post_sample.corr(mu_post_lw)],
    "Spearman": [mu_post_sample.rank().corr(mu_post_lw.rank(), method="spearman")]
}, index=["sample_vs_lw"])
(OUTDIR / "step6_view_stability.tex").write_text(stab.to_latex(escape=False, float_format=lambda x: f"{x: .3f}"))

# Frontier (LW)
def min_var_at_target(mu_target, mu_vec, Sigma, bounds):
    p = len(mu_vec)
    def obj(w): return float(w @ Sigma @ w)
    cons = [
        {"type":"eq", "fun": lambda w: np.sum(w) - 1.0},
        {"type":"eq", "fun": lambda w: float(mu_vec @ w) - float(mu_target)},
    ]
    x0 = np.full(p, 1.0/p)
    return minimize(obj, x0=x0, method="SLSQP", bounds=bounds, constraints=cons,
                    options={"maxiter": 4000, "ftol": 1e-12})

def make_frontier(mu_vec, Sigma, bounds, n=45):
    mu_lo, mu_hi = float(mu_vec.min()), float(mu_vec.max())
    targets = np.linspace(mu_lo*0.5, mu_hi*1.05, n)
    xs, ys = [], []
    for t in targets:
        res = min_var_at_target(t, mu_vec, Sigma, bounds)
        if res.success:
            w = res.x
            xs.append(np.sqrt(float(w @ Sigma @ w)))
            ys.append(float(mu_vec @ w))
    return np.array(xs), np.array(ys)

x_lw, y_lw = make_frontier(mu_post_lw.values, Sigma_lw.loc[assets, assets].values, bounds)

plt.figure(figsize=(6.0,4.4))
plt.plot(x_lw, y_lw, label="Frontier (BL posterior, LW Σ)")
plt.xlabel("Volatility"); plt.ylabel("Expected excess return")
plt.title("Efficient Frontier (LW Σ)")
plt.legend(); plt.tight_layout()
plt.savefig(OUTDIR / "step6_frontier_lw.png"); plt.close()

# MVO (Market) under LW for completeness
def mvo(mu_vec, Sigma, delta, bounds):
    p = len(mu_vec)
    def obj(w): return 0.5*delta*float(w @ Sigma @ w) - float(mu_vec @ w)
    cons = [{"type":"eq", "fun": lambda w: np.sum(w) - 1.0}]
    x0 = np.full(p, 1.0/p)
    return minimize(obj, x0=x0, method="SLSQP", bounds=bounds, constraints=cons,
                    options={"maxiter": 4000, "ftol": 1e-12})

res = mvo(mu_post_lw.values, Sigma_lw.loc[assets, assets].values, delta_market, bounds)
if res.success:
    w_lw_mvo = pd.Series(res.x, index=assets, name="MVO_LW_Market")
    (OUTDIR / "step6_w_MV_lw_Market.tex").write_text(
        (w_lw_mvo*100).to_frame("% Weight").to_latex(escape=False, float_format=lambda x: f"{x: .2f}")
    )

print("Wrote: step6_posterior_mu_lw_tau1overT.tex, step6_frontier_lw.png, step6_w_MV_lw_Market.tex, step6_view_stability.tex")


Wrote: step6_posterior_mu_lw_tau1overT.tex, step6_frontier_lw.png, step6_w_MV_lw_Market.tex, step6_view_stability.tex


In [23]:
# === Step 6 — Cell 11: Risk contributions for BL MVO (Market) and Max Sharpe (sample Σ) ===
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.optimize import minimize
import warnings

OUTDIR = Path("outputs"); OUTDIR.mkdir(parents=True, exist_ok=True)
RET_FILE = OUTDIR / "step2_excess_returns.csv"

# Load returns to rebuild Σ and align assets (keeps this cell standalone)
ret_all = pd.read_csv(RET_FILE, index_col=0, parse_dates=True)
assets = [c for c in ret_all.columns if c in ["NVDA","AMD","ASML","AMAT","TSM","LRCX","INTC","GLD","TLT","VIXY"]]
R = ret_all[assets].copy()
FREQ = 252
Sigma_sample = R.cov() * FREQ

# Reuse posterior μ (τ=1/T) if available; otherwise recompute neutral fallback
try:
    mu_post = posteriors["tau_1_over_T"].loc[assets].values
except NameError:
    mu_post = (R.mean() * FREQ).values  # fallback

# Bounds (same as before)
CAP_SEMI, CAP_GLD_TLT, CAP_VIXY = 0.25, 0.30, 0.05
def make_bounds(assets):
    b = []
    for a in assets:
        if a == "VIXY":
            b.append((0.0, CAP_VIXY))
        elif a in ("GLD","TLT"):
            b.append((0.0, CAP_GLD_TLT))
        else:
            b.append((0.0, CAP_SEMI))
    return tuple(b)
bounds = make_bounds(assets)

Sigma = Sigma_sample.loc[assets, assets].values

def mean_variance(mu, Sigma, delta, bounds):
    p = len(mu)
    def obj(w): return 0.5*delta*float(w @ Sigma @ w) - float(mu @ w)
    cons = [{"type":"eq", "fun": lambda w: np.sum(w) - 1.0}]
    x0 = np.full(p, 1.0/p)
    return minimize(obj, x0=x0, method="SLSQP", bounds=bounds, constraints=cons,
                    options={"maxiter": 4000, "ftol": 1e-12})

def max_sharpe(mu, Sigma, bounds):
    p = len(mu)
    def shp(w):
        num = float(mu @ w); den = np.sqrt(float(w @ Sigma @ w))
        return num/(den+1e-16)
    def obj(w): return -shp(w)
    cons = [{"type":"eq", "fun": lambda w: np.sum(w) - 1.0}]
    x0 = np.full(p, 1.0/p)
    return minimize(obj, x0=x0, method="SLSQP", bounds=bounds, constraints=cons,
                    options={"maxiter": 4000, "ftol": 1e-12})

# Get delta_Market (prefer existing; else compute from posterior μ & Σ)
try:
    delta_market = float(deltas["Market"])
except Exception:
    # quick recompute from posterior μ (consistent with our BL view of the world)
    w_eq = np.full(len(assets), 1.0/len(assets))  # just to get a scale; any reasonable proxy works
    mu_b = float(mu_post @ w_eq)
    sig2_b = float(w_eq @ Sigma @ w_eq)
    delta_market = (mu_b/sig2_b) if sig2_b>0 else 10.0

# Solve weights
res_mvo = mean_variance(mu_post, Sigma, delta_market, bounds)
res_ms  = max_sharpe(mu_post, Sigma, bounds)
if (not res_mvo.success) or (not res_ms.success):
    warnings.warn(f"Optimiser(s) failed. MVO={res_mvo.success}, MS={res_ms.success}")

w_mvo = pd.Series(res_mvo.x, index=assets, name="BL_MVO_Market")
w_ms  = pd.Series(res_ms.x,  index=assets, name="BL_MaxSharpe")

# Risk contributions
def risk_contrib(Sigma, w):
    port_sigma = np.sqrt(float(w @ Sigma @ w))
    mrc = (Sigma @ w) / (port_sigma + 1e-16)        # marginal risk contribution
    prc = (w * mrc) / (port_sigma + 1e-16)          # percentage risk contribution; sums to 1
    return port_sigma, mrc, prc

vol_mvo, mrc_mvo, prc_mvo = risk_contrib(Sigma, w_mvo.values)
vol_ms,  mrc_ms,  prc_ms  = risk_contrib(Sigma, w_ms.values)

rc_mvo = pd.DataFrame({
    "% Weight": w_mvo.values*100,
    "MRC": mrc_mvo,
    "% Risk": prc_mvo*100
}, index=assets)

rc_ms = pd.DataFrame({
    "% Weight": w_ms.values*100,
    "MRC": mrc_ms,
    "% Risk": prc_ms*100
}, index=assets)

# Write LaTeX tables
(OUTDIR / "step6_rc_MV_sample.tex").write_text(rc_mvo.to_latex(escape=False, float_format=lambda x: f"{x: .4f}"))
(OUTDIR / "step6_rc_MS_sample.tex").write_text(rc_ms.to_latex(escape=False, float_format=lambda x: f"{x: .4f}"))

print(f"MVO portfolio vol (annual): {vol_mvo:.4f}")
print(f"MS  portfolio vol (annual): {vol_ms:.4f}")
print("Wrote: step6_rc_MV_sample.tex, step6_rc_MS_sample.tex")


MVO portfolio vol (annual): 0.2922
MS  portfolio vol (annual): 0.2603
Wrote: step6_rc_MV_sample.tex, step6_rc_MS_sample.tex


In [24]:
# === Step 6 — Cell 12: NVDA cap = 30% (others unchanged), re-run MVO (Market) & Max Sharpe ===
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from pathlib import Path

OUTDIR = Path("outputs"); OUTDIR.mkdir(parents=True, exist_ok=True)

# Reuse from earlier cells: assets, Sigma_sample, posteriors, w_mcap
mu_post = posteriors["tau_1_over_T"].loc[assets].values
Sigma_mat = Sigma_sample.loc[assets, assets].values

# --- Per-asset bounds with NVDA at 30% ---
CAP_SEMI, CAP_GLD_TLT, CAP_VIXY = 0.25, 0.30, 0.05
def make_bounds_nvda30(assets):
    b = []
    for a in assets:
        if a == "NVDA":
            b.append((0.0, 0.30))
        elif a == "VIXY":
            b.append((0.0, CAP_VIXY))
        elif a in ("GLD","TLT"):
            b.append((0.0, CAP_GLD_TLT))
        else:
            b.append((0.0, CAP_SEMI))
    return tuple(b)

bounds_nvda30 = make_bounds_nvda30(assets)

def mean_variance(mu, Sigma, delta, bounds):
    p = len(mu)
    def obj(w): return 0.5*delta*float(w @ Sigma @ w) - float(mu @ w)
    cons = [{"type":"eq", "fun": lambda w: np.sum(w) - 1.0}]
    x0 = np.full(p, 1.0/p)
    return minimize(obj, x0=x0, method="SLSQP", bounds=bounds, constraints=cons,
                    options={"maxiter": 4000, "ftol": 1e-12})

def max_sharpe(mu, Sigma, bounds):
    p = len(mu)
    def shp(w):
        num = float(mu @ w); den = np.sqrt(float(w @ Sigma @ w))
        return num/(den+1e-16)
    def obj(w): return -shp(w)
    cons = [{"type":"eq", "fun": lambda w: np.sum(w) - 1.0}]
    x0 = np.full(p, 1.0/p)
    return minimize(obj, x0=x0, method="SLSQP", bounds=bounds, constraints=cons,
                    options={"maxiter": 4000, "ftol": 1e-12})

# Get delta_Market (from memory if available; else recompute)
try:
    delta_market = float(deltas["Market"])
except Exception:
    mu_b = float(mu_post @ w_mcap.loc[assets].values)
    sig2_b = float(w_mcap.loc[assets].values @ Sigma_mat @ w_mcap.loc[assets].values)
    delta_market = (mu_b/sig2_b) if sig2_b>0 else 10.0

# --- Solve with NVDA 30% cap ---
res_mvo = mean_variance(mu_post, Sigma_mat, delta_market, bounds_nvda30)
res_ms  = max_sharpe(mu_post, Sigma_mat, bounds_nvda30)
assert res_mvo.success and res_ms.success, "Optimisers failed under nvda30 bounds."

w_mvo30 = pd.Series(res_mvo.x, index=assets, name="MVO_Market_nvda30")
w_ms30  = pd.Series(res_ms.x,  index=assets, name="MS_Market_nvda30")

# --- Write weights (new filenames) ---
(OUTDIR / "step6_w_MV_sample_Market_nvda30.tex").write_text(
    (w_mvo30*100).to_frame("% Weight").to_latex(escape=False, float_format=lambda x: f"{x: .2f}")
)
(OUTDIR / "step6_w_MS_sample_Market_nvda30.tex").write_text(
    (w_ms30*100).to_frame("% Weight").to_latex(escape=False, float_format=lambda x: f"{x: .2f}")
)

# --- Active vs MCAP (weight-based) metrics + bar charts ---
d_mvo = (w_mvo30 - w_mcap.loc[assets]); d_ms = (w_ms30 - w_mcap.loc[assets])
te_mvo = np.sqrt(float(d_mvo.values @ Sigma_mat @ d_mvo.values))
te_ms  = np.sqrt(float(d_ms.values  @ Sigma_mat @ d_ms.values))
act_mu_mvo = float(mu_post @ d_mvo.values); act_mu_ms = float(mu_post @ d_ms.values)
act_sr_mvo = act_mu_mvo/(te_mvo+1e-16);    act_sr_ms  = act_mu_ms /(te_ms +1e-16)

tbl = pd.DataFrame({
    "Active TE": [te_mvo, te_ms],
    "Active ExpRet": [act_mu_mvo, act_mu_ms],
    "Active Sharpe": [act_sr_mvo, act_sr_ms],
}, index=["MVO_Market_nvda30", "MS_Market_nvda30"])
(OUTDIR / "step6_active_vs_mcap_nvda30.tex").write_text(
    tbl.to_latex(escape=False, float_format=lambda x: f"{x: .4f}")
)

# Active bars
for name, active in [("MVO_Market_nvda30", d_mvo), ("MS_Market_nvda30", d_ms)]:
    plt.figure(figsize=(9,3.4))
    plt.bar(active.index, active.values)
    plt.xticks(rotation=45, ha="right")
    plt.ylabel("Active Weight")
    plt.title(f"Active Weights vs MCAP: {name}")
    plt.tight_layout()
    plt.savefig(OUTDIR / f"step6_active_{name}.png"); plt.close()

print("Wrote: step6_w_MV_sample_Market_nvda30.tex, step6_w_MS_sample_Market_nvda30.tex, step6_active_vs_mcap_nvda30.tex, step6_active_MVO_Market_nvda30.png, step6_active_MS_Market_nvda30.png")


Wrote: step6_w_MV_sample_Market_nvda30.tex, step6_w_MS_sample_Market_nvda30.tex, step6_active_vs_mcap_nvda30.tex, step6_active_MVO_Market_nvda30.png, step6_active_MS_Market_nvda30.png


In [25]:
# === Step 6 — Cell 13: Active vs SMH (return-based) for nvda30 portfolios ===
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

OUTDIR = Path("outputs"); OUTDIR.mkdir(parents=True, exist_ok=True)
RET_FILE = OUTDIR / "step2_excess_returns.csv"

ret_all = pd.read_csv(RET_FILE, index_col=0, parse_dates=True)
assets = [c for c in ret_all.columns if c in ["NVDA","AMD","ASML","AMAT","TSM","LRCX","INTC","GLD","TLT","VIXY"]]
R = ret_all[assets].copy()
FREQ = 252

# Need w_mvo30 and w_ms30; if not present, stop.
try:
    w_mvo30
    w_ms30
except NameError:
    raise RuntimeError("Run Cell 12 first to compute nvda30 weights.")

# Build portfolio daily excess returns (fixed weights)
p_mvo = (R @ w_mvo30).rename("BL_MVO_nvda30")
p_ms  = (R @ w_ms30).rename("BL_MS_nvda30")

# SMH excess series
if "SMH" not in ret_all.columns:
    print("SMH not found in returns file; skipping SMH active metrics.")
else:
    smh = ret_all["SMH"]
    active_mvo = (p_mvo - smh).dropna()
    active_ms  = (p_ms  - smh).dropna()

    # Annualized TE and IR (return-based)
    te_mvo = active_mvo.std()*np.sqrt(FREQ)
    te_ms  = active_ms.std()*np.sqrt(FREQ)
    ir_mvo = active_mvo.mean()*FREQ / (te_mvo + 1e-16)
    ir_ms  = active_ms.mean()*FREQ / (te_ms  + 1e-16)

    df = pd.DataFrame({
        "TE (ann)": [te_mvo, te_ms],
        "IR":       [ir_mvo, ir_ms],
        "Ann Active Ret (%)": [active_mvo.mean()*FREQ*100, active_ms.mean()*FREQ*100],
    }, index=["BL_MVO_nvda30","BL_MS_nvda30"])

    (OUTDIR / "step6_active_vs_smh_nvda30.tex").write_text(
        df.to_latex(escape=False, float_format=lambda x: f"{x: .3f}")
    )

    # cumulative active P&L
    plt.figure(figsize=(8.0,4.0))
    (1+active_mvo).cumprod().plot(label="BL_MVO_nvda30 - SMH")
    (1+active_ms).cumprod().plot(label="BL_MS_nvda30 - SMH")
    plt.title("Cumulative Active P&L vs SMH (No Rebalancing)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(OUTDIR / "step6_active_cum_vs_smh_nvda30.png"); plt.close()

    print("Wrote: step6_active_vs_smh_nvda30.tex, step6_active_cum_vs_smh_nvda30.png")


Wrote: step6_active_vs_smh_nvda30.tex, step6_active_cum_vs_smh_nvda30.png


In [27]:
import nbformat
from pathlib import Path

nb_path = Path("/Users/dave/PythonProject1/PythonProject1/PythonProject/BlackLitterman/BlackLitterman.ipynb")
nb = nbformat.read(nb_path, as_version=4)

raw = nonempty = code_only = 0
for cell in nb.cells:
    if cell.cell_type != "code":
        continue
    for line in cell.source.splitlines():
        raw += 1
        s = line.strip()
        if s:
            nonempty += 1
        if s and not s.startswith(('#','%','!')):
            code_only += 1

print(f"Notebook: {nb_path.name}")
print(f" Raw lines   : {raw}")
print(f" Non-empty   : {nonempty}")
print(f" Code only   : {code_only}")


Notebook: BlackLitterman.ipynb
 Raw lines   : 2376
 Non-empty   : 2026
 Code only   : 1748
